# 1. 기본 환경 설정

In [1]:
# 필요 라이브러리 모음
import os
from tqdm import tqdm
import requests

import numpy as np
import pandas as pd
import re

import warnings
warnings.filterwarnings('ignore')

# 좌표 간 거리 계산을 위해
from haversine import haversine, Unit

Global Variables

In [2]:
import os
import easydict
args = easydict.EasyDict()

args.default_path = 'C:/python/Budongsan/new_data/'

#해당 폴더 안에 있는 파일 리스트 불러오기
data_list = os.listdir(args.default_path)
print(data_list)

['.ipynb_checkpoints', '5대_범죄_발생현황.csv', 'Analysis_of_interim_results.ipynb', 'apt_master_20230730.csv', 'apt_Prediction_Evaluation.csv', 'apt_sale_price.csv', 'apt_sale_price_f.csv', 'apt_sale_price_f1.csv', 'apt_sale_price_f2.csv', 'apt_sale_price_f3.csv', 'apt_sale_price_main_nan.csv', 'apt_sale_price_main_nan_f.csv', 'business_modified_bubjeong_df.csv', 'busstop_name_addr_count.csv', 'busstop_name_addr_count_main.csv', 'coordinate_data.json', 'final_master.csv', 'gwr_use', 'master_data', 'price_code.xlsx', 'real_final_master_data.csv', 'subway189.csv', 'subway189_nan.csv', 'Total.JUSUAN.20230701.TI_SPOT_BUSST_ADRES.TXT', 'Total.JUSUAN.20230701.TI_SPOT_BUSST_M.TXT', '교통_인프라_요인', '기타_요인', '도로현황(도로율)_20230703233927.csv', '부동산_매매_월별_키워드_검색어트렌드.csv', '부동산_주거_시장_요인', '서울_행정동코드.csv', '서울교통공사 노선별 지하철역 정보.csv', '서울교통공사_역사정보.csv', '서울메트로9호선_도시철도역사정보.csv', '서울시 주요 공원현황.csv', '서울시_동별_초중고대_정보.csv', '서울시버스정류소위치정보(20230613).xlsx', '소비심리지수.csv', '아파트(매매)__실거래가_2018.csv', '아파트(매매)__실거래가_2019.csv', 

필요 함수 정리

In [3]:
def chk_unique(df):
    ''' 데이터의 범주형 컬럼별 고유값 확인 '''
    for c in df.select_dtypes(exclude = np.number).columns.tolist():
        print('\n######',c )
        print(df[c].unique())

def chg_date(df, col, format:str):
    '''날짜 데이터를 datetime 형식으로 변경 후 년도 컬럼 생성'''
    if type(df[col]) != 'string':
        df[col] = pd.to_datetime(df[col].astype('str'), format=format)
    else:
        # df['년도'] = df[col].dt.year
        df[col] = pd.to_datetime(df[col], format=format)

def chk_df(df):
    print(df.shape)
    print(df.info())
    print(df.head())

def prep_october(col):
    ''' 10월의 데이터를 .1 표시에서 .10로 변경'''
    if len(col) == 6:
        col = col + '0'
        return col

# 2. 데이터 불러오기

In [4]:
args.default_path

'C:/python/Budongsan/new_data/'

Key Data load

In [5]:
price_code = pd.read_excel(args.default_path + 'price_code.xlsx')

### Load apt price

In [6]:
list_of_dir = []

for name in data_list:
    if '아파트(매매)__실거래가' in name:
        df = pd.read_csv(args.default_path + name , encoding = 'cp949')
        list_of_dir.append(df)

apt_sale_price = pd.concat(list_of_dir)
# apt_sale_price.head()

#자치구와 법정동
si, gu, dong = apt_sale_price['시군구'].str.split(' ').str
apt_sale_price['시'] = si
apt_sale_price['자치구'] = gu
apt_sale_price['법정동'] = dong

# to_datetime
chg_date(apt_sale_price,'계약년월','%Y%m')
apt_sale_price['연도'] = apt_sale_price['계약년월'].dt.year.astype('object')
apt_sale_price['월'] = apt_sale_price['계약년월'].dt.month.astype('object')

# 도로명주소
apt_sale_price['도로명주소'] = apt_sale_price['시'] + ' ' + apt_sale_price['자치구'] + ' ' + apt_sale_price['도로명']

# 지번주소
apt_sale_price['지번주소'] = apt_sale_price['시군구'] + ' ' + apt_sale_price['번지']

# 계약년월 -> 시점으로 컬럼명 변경
apt_sale_price.rename(columns = {'계약년월' : '시점'},inplace = True)

chk_df(apt_sale_price) # 데이터 확인

(297028, 22)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 297028 entries, 0 to 12683
Data columns (total 22 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   시군구       297028 non-null  object        
 1   번지        296920 non-null  object        
 2   본번        296985 non-null  float64       
 3   부번        296985 non-null  float64       
 4   단지명       297028 non-null  object        
 5   전용면적(㎡)   297028 non-null  float64       
 6   시점        297028 non-null  datetime64[ns]
 7   계약일       297028 non-null  int64         
 8   거래금액(만원)  297028 non-null  object        
 9   층         297028 non-null  int64         
 10  건축년도      297026 non-null  float64       
 11  도로명       297028 non-null  object        
 12  해제사유발생일   5356 non-null    float64       
 13  거래유형      297028 non-null  object        
 14  중개사소재지    297028 non-null  object        
 15  시         297028 non-null  object        
 16  자치구       297028 non-null 

In [7]:
apt_sale_price.columns

Index(['시군구', '번지', '본번', '부번', '단지명', '전용면적(㎡)', '시점', '계약일', '거래금액(만원)', '층',
       '건축년도', '도로명', '해제사유발생일', '거래유형', '중개사소재지', '시', '자치구', '법정동', '연도',
       '월', '도로명주소', '지번주소'],
      dtype='object')

In [8]:
apt_sale_price = apt_sale_price[['시군구', '번지', '본번', '부번', '단지명', '전용면적(㎡)', '시점', '계약일', '거래금액(만원)', '층',
       '건축년도', '도로명', '시', '자치구', '법정동', '연도', '월', '도로명주소', '지번주소']]

In [9]:
apt_sale_price.to_csv('C:/python/Budongsan/new_data/apt_sale_price.csv', encoding='utf-8')

#### 좌표 붙인 뒤

In [10]:
apt_sale_price_main = pd.read_csv(args.default_path + 'apt_sale_price_f.csv')
apt_sale_price_f1 = pd.read_csv(args.default_path + 'apt_sale_price_f1.csv')
apt_sale_price_f2 = pd.read_csv(args.default_path + 'apt_sale_price_f2.csv')
apt_sale_price_f3 = pd.read_csv(args.default_path + 'apt_sale_price_f3.csv')

# 좌표 첫 시도에 안 붙은 경우 새로 붙여주기
apt_sale_price_f1 = apt_sale_price_f1[['key','x','y']]
apt_sale_price_f2 = apt_sale_price_f2[['key','x','y']]
apt_sale_price_f3 = apt_sale_price_f3[['key','x','y']]

# 좌표 2차 시도에 안 붙은 경우 새로 붙여주기
apt_sale_price_nan= pd.read_csv(args.default_path + 'apt_sale_price_main_nan_f.csv')

apt_sale_price_nan = apt_sale_price_nan[['key','x','y']]

# 안된 데이터들 한번에 붙여주기
apt_sale_price_f = pd.concat([apt_sale_price_f1, apt_sale_price_f2, apt_sale_price_f3, apt_sale_price_nan], axis = 0)

# 첫 시도한 경우랑 모두 합쳐서 마무리
apt_sale_price_main = pd.merge(apt_sale_price_main, apt_sale_price_f, left_on=['key'], right_on=['key'], how='left')

# 사용할 column만 남기기
apt_sale_price_main = apt_sale_price_main[['시군구', '번지', '본번', '부번', '단지명', '전용면적(㎡)', '시점', '계약일', '거래금액(만원)', '층',
       '건축년도', '도로명', '시', '자치구', '법정동', '연도', '월', '도로명주소', '지번주소', 'x', 'y']]

# 용량 최소화를 위한 type 변경
apt_sale_price_main['x'] = apt_sale_price_main['x'].astype('float32')
apt_sale_price_main['y'] = apt_sale_price_main['y'].astype('float32')

chk_df(apt_sale_price_main) # 데이터 확인

(297028, 21)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 297028 entries, 0 to 297027
Data columns (total 21 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   시군구       297028 non-null  object 
 1   번지        296920 non-null  object 
 2   본번        296985 non-null  float64
 3   부번        296985 non-null  float64
 4   단지명       297028 non-null  object 
 5   전용면적(㎡)   297028 non-null  float64
 6   시점        297028 non-null  object 
 7   계약일       297028 non-null  int64  
 8   거래금액(만원)  297028 non-null  object 
 9   층         297028 non-null  int64  
 10  건축년도      297026 non-null  float64
 11  도로명       296901 non-null  object 
 12  시         297028 non-null  object 
 13  자치구       297028 non-null  object 
 14  법정동       297028 non-null  object 
 15  연도        297028 non-null  int64  
 16  월         297028 non-null  int64  
 17  도로명주소     297028 non-null  object 
 18  지번주소      296920 non-null  object 
 19  x         297028 non-null  floa

## 1) 인구 사회 요인

### 인구이동

In [11]:
pop_movement = pd.read_csv(args.default_path + '인구이동(2015년+이후).csv')
# pop_movement.info()

chk_unique(pop_movement)

#불필요한 컬럼 제거 - 동별(1), 순이동(.2)
pop_movement.drop(['동별(1)','2021.2','2022.2','성별(1)'],axis =1 ,inplace=True)

# 첫번째 분류 데이터 제거()
pop_movement = pop_movement.iloc[1:].reset_index(drop=True)
pop_movement.head()

# 컬러명 변경
pop_movement.rename(columns={'동별(2)':'자치구','동별(3)':'행정동'} ,inplace =True )
pop_movement

# 결측치 처리 ('-' 표시 0으로 변경)
pop_movement.replace('-',0)

# #중복값 확인
pop_movement[pop_movement[['자치구','행정동']].duplicated(keep=False)]

# melt - 전입 전출 구분
in_cols = ['2018','2019','2020','2021','2022']
out_cols = ['2018.1','2019.1','2020.1','2021.1','2022.1']

in_df = pd.melt(pop_movement,id_vars = ['자치구','행정동'],value_vars= in_cols, var_name='연도', value_name='총전입')
out_df = pd.melt(pop_movement,id_vars = ['자치구','행정동'],value_vars= out_cols, var_name='연도', value_name='총전출')

out_df['연도'] = out_df['연도'].str.split('.').str[0]

print(in_df.shape)
print(out_df.shape)

pop_move = pd.merge(in_df,out_df,how='outer')
pop_move.shape
pop_move.head()


###### 동별(1)
['동별(1)' '합계']

###### 동별(2)
['동별(2)' '종로구' '중구' '용산구' '성동구' '광진구' '동대문구' '중랑구' '성북구' '강북구' '도봉구' '노원구'
 '은평구' '서대문구' '마포구' '양천구' '강서구' '구로구' '금천구' '영등포구' '동작구' '관악구' '서초구' '강남구'
 '송파구' '강동구']

###### 동별(3)
['동별(3)' '사직동' '삼청동' '부암동' '평창동' '무악동' '교남동' '가회동' '종로1.2.3.4가동' '종로5.6가동'
 '이화동' '창신1동' '창신2동' '창신3동' '숭인1동' '숭인2동' '청운효자동' '혜화동' '소공동' '회현동' '명동'
 '필동' '장충동' '광희동' '을지로동' '신당5동' '황학동' '중림동' '신당동' '다산동' '약수동' '청구동' '동화동'
 '후암동' '용산2가동' '남영동' '원효로2동' '효창동' '용문동' '이촌1동' '이촌2동' '이태원1동' '이태원2동'
 '서빙고동' '보광동' '청파동' '원효로1동' '한강로동' '한남동' '왕십리2동' '마장동' '사근동' '행당1동' '행당2동'
 '응봉동' '금호1가동' '금호4가동' '성수1가1동' '성수1가2동' '성수2가1동' '성수2가3동' '송정동' '용답동'
 '왕십리도선동' '금호2.3가동' '옥수동' '화양동' '군자동' '중곡1동' '중곡2동' '중곡3동' '중곡4동' '능동'
 '구의1동' '구의2동' '구의3동' '광장동' '자양1동' '자양2동' '자양3동' '자양4동' '회기동' '휘경1동'
 '휘경2동' '청량리동' '용신동' '제기동' '전농1동' '전농2동' '답십리2동' '장안1동' '장안2동' '이문1동'
 '이문2동' '답십리1동' '면목2동' '면목4동' '면목5동' '면목7동' '상봉1동' '상봉2동' '중화1동' '중화2동'
 '묵1동' '묵2동' '망우3동' '신내1동' '신내2동' '면목본동' '면목3.8동' '망우본동' '

,자치구,행정동,연도,총전입,총전출
0,종로구,사직동,2018,1802,1685
1,종로구,삼청동,2018,363,431
2,종로구,부암동,2018,1306,1444
3,종로구,평창동,2018,2576,2623
4,종로구,무악동,2018,935,1021


### 주민등록인구

In [12]:
pop_ages = pd.read_csv(args.default_path + '주민등록인구_20230713231331.csv' , encoding ='utf-8')

# 시점 -> 연도
pop_ages.rename(columns = {'시점':'연도'},inplace = True)

chk_df(pop_ages) # 데이터 확인

(2125, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2125 entries, 0 to 2124
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   연도       2125 non-null   int64 
 1   자치구      2125 non-null   object
 2   행정동      2125 non-null   object
 3   주민등록인구수  2125 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 66.5+ KB
None
     연도  자치구  행정동  주민등록인구수
0  2018  종로구  사직동     9818
1  2018  종로구  삼청동     3073
2  2018  종로구  부암동    10537
3  2018  종로구  평창동    19105
4  2018  종로구  무악동     8260


### 생활인구

In [13]:
life_people = pd.read_csv(args.default_path + '자치구단위 서울생활인구 일별 집계표.csv' , encoding ='utf-8')

# 시군구코드와 시군구명
sigungu = pd.read_csv('./Budongsan/new_data/서울_행정동코드.csv',index_col=0)
sigungu.reset_index(inplace=True)

# column명 변경
life_people = life_people.rename(columns={'기준일':'시점'})

life_people_merge = pd.merge(life_people,sigungu, left_on='자치구코드', right_on='자치구코드', how="inner")

# 구별 일평균
life_people_merge_gu_hour = life_people_merge.groupby(['시군구명','시점'], as_index=False).mean()

#to_datetime
chg_date(life_people_merge_gu_hour,'시점','%Y%m%d')
life_people_merge_gu_hour['연도'] = life_people_merge_gu_hour['시점'].dt.year
life_people_merge_gu_hour['월'] = life_people_merge_gu_hour['시점'].dt.month

# 구별 월 평균
life_people_mean = life_people_merge_gu_hour.groupby(['시군구명','연도','월'])['총생활인구수'].mean()
life_people_mean = life_people_mean.reset_index(drop=False)

chk_df(life_people_mean) # 데이터 확인

(1500, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   시군구명    1500 non-null   object 
 1   연도      1500 non-null   int64  
 2   월       1500 non-null   int64  
 3   총생활인구수  1500 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 47.0+ KB
None
  시군구명    연도  월         총생활인구수
0  강남구  2018  1  837615.719594
1  강남구  2018  2  803034.101304
2  강남구  2018  3  826003.757742
3  강남구  2018  4  817527.898616
4  강남구  2018  5  805135.717920


## 2) 교통 인프라 요인

### 서울시 버스정류장 현황

In [14]:
# 버스정류장 이름 데이터 불러오기
busstop_name = pd.read_csv(args.default_path + 'Total.JUSUAN.20230701.TI_SPOT_BUSST_M.txt', sep='|', names=['정류장 ID', '정류소명', '영문명', '시군구코드', '시', '구', '변경일자', '변경사유코드'], header=None, encoding='cp949')

# 버스정류장 이름 데이터 전처리
busstop_name.reset_index(drop=False, inplace=True)
busstop_name = busstop_name.rename(columns={'index':'정류장 ID', '정류장 ID':'정류소명', '정류소명':'A', '영문명':'시군구코드', '시군구코드':'시', '시':'구', '구':'구1'})

busstop_name = busstop_name[['정류장 ID', '정류소명', '시']]

busstop_name = busstop_name[busstop_name['시'] == '서울특별시']
busstop_name = busstop_name[['정류장 ID', '정류소명']]

# 버스정류장 주소 데이터 불러오기
busstop_addr = pd.read_csv(args.default_path + 'Total.JUSUAN.20230701.TI_SPOT_BUSST_ADRES.txt', sep='|', names=['정류장 ID', 'A', 'B', '시군구코드', '시', '구', '동', '도로명코드', '도로명', '지하여부', '주소본번', '주소부번', '행정구역코드', '대표주소여부', '변경일자' , '변경사유코드'], encoding='cp949')

# 주소 데이터 중 본,부번 타입 변경(주소 활용 위해)
busstop_addr['주소본번'] = busstop_addr['주소본번'].astype(str)
busstop_addr['주소부번'] = busstop_addr['주소부번'].astype(str)

# 주소 데이터 전처리
busstop_addr = busstop_addr[busstop_addr['시'] == '서울특별시']
## 도로명주소
busstop_addr['도로명주소'] = busstop_addr['시'] + ' ' + busstop_addr['구'] + ' ' + busstop_addr['동'] + ' ' + busstop_addr['도로명']
## 지번주소
busstop_addr['지번주소'] = busstop_addr['시'] + ' ' + busstop_addr['구'] + ' ' + busstop_addr['동'] + ' ' + busstop_addr['주소본번'] + '-' + busstop_addr['주소부번']
busstop_addr = busstop_addr[['정류장 ID', '시군구코드', '시', '구', '동', '도로명코드', '도로명', '행정구역코드', '도로명주소', '지번주소']]

# 이름 데이터와 주소 데이터 합치기
busstop_name_addr = pd.merge(busstop_name, busstop_addr, on=['정류장 ID'], how='left')

# nan 값 해결 - 오류 확인
busstop_name_addr = busstop_name_addr[busstop_name_addr['동'].notnull()]

# 동별 버스정류장 수 계산 후 merge
busstop_name_addr_s = busstop_name_addr.groupby('동')['동'].count().reset_index(name='count')

busstop_name_addr_count = pd.merge(busstop_name_addr, busstop_name_addr_s, on=['동'], how='left')

chk_df(busstop_name_addr_count) # 데이터 확인

(10636, 12)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10636 entries, 0 to 10635
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   정류장 ID  10636 non-null  object 
 1   정류소명    10636 non-null  object 
 2   시군구코드   10636 non-null  float64
 3   시       10636 non-null  object 
 4   구       10636 non-null  object 
 5   동       10636 non-null  object 
 6   도로명코드   10636 non-null  float64
 7   도로명     10636 non-null  object 
 8   행정구역코드  10636 non-null  float64
 9   도로명주소   10636 non-null  object 
 10  지번주소    10636 non-null  object 
 11  count   10636 non-null  int64  
dtypes: float64(3), int64(1), object(8)
memory usage: 1.1+ MB
None
               정류장 ID          정류소명    시군구코드      시    구      동      도로명코드  \
0  OBJ030000000000936       마장축산물시장  11200.0  서울특별시  성동구    마장동  3005030.0   
1  OBJ030000000001052  성수동SK아파트형공장앞  11200.0  서울특별시  성동구  성수동1가  3103004.0   
2  OBJ030000000001113    힐스테이트서울숲리버  11200.0  서울특별시  성동구  금호

In [15]:
# busstop_name_addr_count.to_csv('C:/python/Budongsan/new_data/busstop_name_addr_count.csv', encoding='utf-8')

#### 좌표 추가로 붙인 뒤에

In [16]:
busstop_name_addr_count_main = pd.read_csv(args.default_path + 'busstop_name_addr_count_main.csv')

busstop_name_addr_count_main['x'] = busstop_name_addr_count_main['x'].astype('float32')
busstop_name_addr_count_main['y'] = busstop_name_addr_count_main['y'].astype('float32')

chk_df(busstop_name_addr_count_main) # 데이터 확인

(10636, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10636 entries, 0 to 10635
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   정류장 ID  10636 non-null  object 
 1   정류소명    10636 non-null  object 
 2   시군구코드   10636 non-null  int64  
 3   시       10636 non-null  object 
 4   구       10636 non-null  object 
 5   동       10636 non-null  object 
 6   도로명코드   10636 non-null  int64  
 7   도로명     10636 non-null  object 
 8   행정구역코드  10636 non-null  int64  
 9   도로명주소   10636 non-null  object 
 10  지번주소    10636 non-null  object 
 11  count   10636 non-null  int64  
 12  x       10636 non-null  float32
 13  y       10636 non-null  float32
dtypes: float32(2), int64(4), object(8)
memory usage: 1.1+ MB
None
               정류장 ID          정류소명  시군구코드      시    구      동    도로명코드   도로명  \
0  OBJ030000000000936       마장축산물시장  11200  서울특별시  성동구    마장동  3005030  고산자로   
1  OBJ030000000001052  성수동SK아파트형공장앞  11200  서울특별시  성동구  성수동1

### 서울시 자하철역 정보

In [17]:
#1~8호선
subway18=pd.read_csv(args.default_path + '서울교통공사_역사정보.csv', encoding ='utf-8')
#9호선
subway9=pd.read_csv(args.default_path + '서울메트로9호선_도시철도역사정보.csv', encoding ='utf-8')
#우이신설선
ui=pd.read_csv(args.default_path + '우이신설경전철_역사정보_20230628.csv', encoding ='utf-8')

In [25]:
#1~8호선
##컬럼 복사해서 split
subway18['역 주소(도로명 주소)2']=subway18['역 주소(도로명 주소)'].apply(lambda x: x.split('('))
##동만 추출하고 뒤에 있는 괄호 ')' 삭제
subway18['동']=subway18['역 주소(도로명 주소)2'].str[1]
subway18['동']=subway18['동'].astype(str).apply(lambda x: x[:-1])

#9호선
subway9['역사도로명주소2']=subway9['역사도로명주소'].apply(lambda x: x.split('('))
subway9['동']=subway9['역사도로명주소2'].str[1]
subway9['동']=subway9['동'].astype(str).apply(lambda x: x[:-1])

# merge
## 칼람먕 일치
subway18.rename(columns={'역명(한글)':'역사명'}, inplace=True)

subway189=pd.merge(subway18,subway9,how='outer')

## 1차 컬럼 drop
subway189.drop(['역 주소(도로명 주소)2','역사도로명주소2','No', '신설일자','폐지일자', '상행거리', '하행거리','데이터 기준일자','데이터기준일자','역명(영어)', '역명(부역명)','역사 전화번호','상행거리', '하행거리', '참고사항'], axis=1, inplace=True)

## 1~8호선 컬럼 일치시켜주기
subway189['역 주소(도로명 주소)'] = subway189['역 주소(도로명 주소)'].fillna(' ')
subway189['역사도로명주소'] = subway189['역사도로명주소'].fillna(' ')
subway189['도로명주소'] = subway189['역 주소(도로명 주소)'].add(subway189['역사도로명주소'])

subway189['역 번호'] = subway189['역 번호'].astype('str').fillna(' ')
subway189['역번호'] = subway189['역번호'].astype('str').fillna(' ')
subway189['역_변호'] = subway189['역 번호'].add(subway189['역번호'])

subway189['운영노선'] = subway189['운영노선'].fillna(' ')
subway189['노선명'] = subway189['노선명'].fillna(' ')
subway189['호선'] = subway189['운영노선'].add(subway189['노선명'])

#2차 컬럼 drop
subway189.drop(['역 주소(도로명 주소)', '역사도로명주소','철도운영기관명','역 종류','역 번호','역번호','역위도','역경도','노선번호','노선명','운영노선','환승노선번호','환승역구분','승강장 유형','승강장 연결여부', '역_변호'], axis=1, inplace=True)
subway189['환승역 여부']=subway189['환승역 여부'].fillna('X')

## 우이신설선 merge
subway189=pd.concat([subway189, ui], ignore_index=True)

#컬럼 일치시키기
subway189['도로명주소'] = subway189['도로명주소'].fillna('') + subway189['역 주소(도로명 주소)'].fillna('')
subway189['호선'] = subway189['호선'].fillna('') + subway189['운영노선'].fillna('')
subway189['역사명'] = subway189['역사명'].fillna('') + subway189['역명(한글)'].fillna('')

#필요없는 컬럼 제거
subway189.drop(columns=['운영노선', '역명(한글)', '역 주소(도로명 주소)'], inplace=True)

#역사명 서울에서 서울역으로 변경
subway189.loc[subway189['역사명'] == '서울', '역사명'] = '서울역'

#####여기까지 코드로 하고 나머지 csv로 추출한것에 수인분당선, 신분당선, 경의중앙, 공항철도, 경춘선, 신림선 모두 서울에 위치한 역들만 골라서 엑셀로 진행 #####

In [18]:
# 최종 지하철 df
subway189=pd.read_csv(args.default_path + 'subway189.csv')

# 필요 컬럼만 남기기
subway189 = subway189[['역사명', '역 위치(경도)', '역 위치(위도)', '역 주소(지번주소)', '도로명주소', '시군구명','법정동명','법정동코드']]

# null값 채워주기
subway189.loc[383, '법정동명'] = '신길동'

# subway189_nan.to_csv('C:/python/Budongsan/new_data/subway189_nan.csv', encoding='utf-8')
### 좌표 nan값 채워준뒤 

# 동별 지하철역 수 계산
subway_dong = subway189[['시군구명','법정동명','법정동코드']].value_counts().reset_index()
subway_dong

# count 데이터 합치기
subway189_count = pd.merge(subway189, subway_dong, on=['시군구명','법정동명','법정동코드'], how='left')

# count column 이름 변경
subway189_count = subway189_count.rename(columns={0:'지하철역 수'})

# 용량 효율 위해 타입 변경
subway189_count['역 위치(경도)'] = subway189_count['역 위치(경도)'].astype('float32')
subway189_count['역 위치(위도)'] = subway189_count['역 위치(위도)'].astype('float32')

chk_df(subway189_count) # 데이터 확인

(393, 9)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 393 entries, 0 to 392
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   역사명         393 non-null    object 
 1   역 위치(경도)    393 non-null    float32
 2   역 위치(위도)    393 non-null    float32
 3   역 주소(지번주소)  393 non-null    object 
 4   도로명주소       393 non-null    object 
 5   시군구명        393 non-null    object 
 6   법정동명        393 non-null    object 
 7   법정동코드       393 non-null    int64  
 8   지하철역 수      393 non-null    int64  
dtypes: float32(2), int64(2), object(5)
memory usage: 27.6+ KB
None
    역사명    역 위치(경도)   역 위치(위도)                      역 주소(지번주소)  \
0   서울역  126.972534  37.553150   서울특별시 중구 남대문로5가 73-6 서울역(1호선)   
1    시청  126.975410  37.563591        서울특별시 중구 정동 5-5 시청역(1호선)   
2    종각  126.983116  37.570202      서울특별시 종로구 종로1가 54 종각역(1호선)   
3  종로3가  126.992096  37.570431  서울특별시 종로구 종로3가 10-5 종로3가역(1호선)   
4  종로5가  127.001900  37.570972  서울특별시 

### 도로율

In [19]:
road = pd.read_csv(args.default_path + '도로현황(도로율)_20230703233927.csv', encoding ='utf-8')

# 여러 컬럼 뽑아오기
selected_columns = ['자치구별(2)', '2018.2', '2019.2', '2020.2', '2021.2', '2022.2']
road = road[selected_columns]

# column명 변경
road = road.rename(columns={'자치구별(2)':'시군구', '2018.2':2018, '2019.2':2019, '2020.2':2020, '2021.2':2021, '2022.2':2022})
road = road.iloc[2:]

# melt
road = road.melt(id_vars='시군구',
                        value_vars=[2018, 2019, 2020, 2021, 2022],
                         var_name='연도',
                         value_name='도로율')

chk_df(road) # 데이터 확인

(125, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시군구     125 non-null    object
 1   연도      125 non-null    object
 2   도로율     125 non-null    object
dtypes: object(3)
memory usage: 3.1+ KB
None
   시군구    연도    도로율
0  종로구  2018  21.34
1   중구  2018  18.82
2  용산구  2018  22.15
3  성동구  2018  23.09
4  광진구  2018  28.50


## 3) 부동산 주거 시장 요인

### 지역 내 총생산

In [20]:
grdp = pd.read_excel(args.default_path + '자치구별+지역내총생산(2015년+기준)_20230704003019.xlsx')

# 여러 컬럼 뽑아오기
selected_columns = ['자치구', 2018, 2019, 2020, 2021, 2022]
grdp = grdp[selected_columns]

# melt
grdp = grdp.melt(id_vars='자치구',
                        value_vars=[2018, 2019, 2020, 2021, 2022],
                         var_name='연도',
                         value_name='지역내총생산')

chk_df(grdp) # 데이터 확인

(125, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   자치구     125 non-null    object
 1   연도      125 non-null    object
 2   지역내총생산  125 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 3.1+ KB
None
   자치구    연도    지역내총생산
0  종로구  2018  31868698
1   중구  2018  54274488
2  용산구  2018  12300317
3  성동구  2018  11657151
4  광진구  2018   6342279


### 지역별 월평균 소득

In [21]:
gri = pd.read_csv(args.default_path + '읍면동별 월평균 가구소득.csv', encoding ='utf-8')

# 여러 컬럼 뽑아오기
selected_columns = ['기준년코드', '행정동코드', '행정동코드명', '월평균소득금액']
gri = gri[selected_columns]

# column명 변경
gri = gri.rename(columns={'기준년코드':'연도', '행정동코드명':'행정동'})

# merge를 위한 값 변경
gri.loc[gri['행정동'] == '금호2·3가동', '행정동'] = '금호2.3가동'
gri.loc[gri['행정동'] == '면목제3·8동', '행정동'] = '면목제3.8동'
gri.loc[gri['행정동'] == '상계3·4동', '행정동'] = '상계3.4동'
gri.loc[gri['행정동'] == '상계6·7동', '행정동'] = '상계6.7동'
gri.loc[gri['행정동'] == '종로1·2·3·4가동', '행정동'] = '종로1.2.3.4가동'
gri.loc[gri['행정동'] == '종로5·6가동', '행정동'] = '종로5.6가동'
gri.loc[gri['행정동'] == '중계2·3동', '행정동'] = '중계2.3동'

# 조건 인덱싱을 사용하여 특정 조건을 만족하는 행 추출
gri = gri[(gri['연도'] >= 2018)]

chk_df(gri) # 데이터 확인

(2121, 4)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2121 entries, 1272 to 3392
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   연도       2121 non-null   int64 
 1   행정동코드    2121 non-null   int64 
 2   행정동      2121 non-null   object
 3   월평균소득금액  2121 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 82.9+ KB
None
        연도     행정동코드    행정동  월평균소득금액
1272  2018  11110515  청운효자동  4155842
1273  2018  11110530    사직동  5247480
1274  2018  11110540    삼청동  3880276
1275  2018  11110550    부암동  3856181
1276  2018  11110560    평창동  4896738


### 기준금리

In [22]:
ir = pd.read_csv(args.default_path + '한국은행 기준금리_연도별.csv', encoding ='utf-8')

# 행 열 전환
ir=ir.transpose()

# column명 변경
ir.rename(columns=ir.iloc[0],inplace=True)
ir = ir.drop(ir.index[0])
ir = ir[5:10]

ir.reset_index(drop=False, inplace=True)
ir = ir.rename(columns={'index': '연도', '1.3.1. 한국은행 기준금리 및 여수신금리':'기준금리'})

chk_df(ir) # 데이터 확인

(5, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   연도      5 non-null      object
 1   기준금리    5 non-null      object
dtypes: object(2)
memory usage: 212.0+ bytes
None
     연도  기준금리
0  2018  1.75
1  2019  1.25
2  2020   0.5
3  2021   1.0
4  2022  3.25


### 물가

In [23]:
cpi = pd.read_csv(args.default_path + '지출목적별_소비자물가지수_품목포함_월별_2016_2022.csv', encoding ='cp949')

# 총 지수만 추출
cpi = cpi[(cpi['지출목적별'] == '0 총지수')]

# 행 열 전환
cpi=cpi.transpose()

# column명 변경
cpi.rename(columns=cpi.iloc[0],inplace=True)
cpi = cpi.drop(cpi.index[0])

cpi.reset_index(drop=False, inplace=True)
cpi = cpi[1:84]
cpi = cpi.rename(columns={'index':'시점','전국':'소비자물가지수'})

#to_datetime
chg_date(cpi,'시점','%Y.%m')
cpi['연도'] = cpi['시점'].dt.year
cpi['월'] = cpi['시점'].dt.month

# 월별 데이터의 연평균 계산
cpi_mean = cpi.groupby(['연도','월'])['소비자물가지수'].mean()

# 데이터 정리
cpi_mean = cpi_mean.reset_index(drop=False)

chk_df(cpi_mean) # 데이터 확인

(83, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   연도       83 non-null     int64  
 1   월        83 non-null     int64  
 2   소비자물가지수  83 non-null     float64
dtypes: float64(1), int64(2)
memory usage: 2.1 KB
None
     연도  월  소비자물가지수
0  2016  1   95.232
1  2016  2   95.640
2  2016  3   95.393
3  2016  4   95.573
4  2016  5   95.630


### 전세가율

In [24]:
sale_jeonse_rt = pd.read_excel(args.default_path + '유형별_매매가격_대비_전세가격_비율_2018_2022.xlsx')
# sale_jeonse_rt.head()
chk_unique(sale_jeonse_rt) #고유값 확인

#시점 전처리 (10월)
# sale_jeonse_rt['시점'] = sale_jeonse_rt['시점'].astype('string').apply(prep_october)

#to_datetime
chg_date(sale_jeonse_rt,'시점','%Y-%m')

sale_jeonse_rt.drop(['지역별(1)', '지역별(2)', '지역별(3)', '주택유형별(1)'],axis = 1 , inplace =True)
sale_jeonse_rt.rename(columns = {'지역별(4)': '자치구','데이터':'전세가율'},inplace = True)

# 자치구 전처리
sale_jeonse_rt['자치구'] = sale_jeonse_rt['자치구'] + '구'

# '월' 열 추가
sale_jeonse_rt['연도'] = sale_jeonse_rt['시점'].dt.year
sale_jeonse_rt['월'] = sale_jeonse_rt['시점'].dt.month

chk_df(sale_jeonse_rt) # 데이터 확인


###### 주택유형별(1)
['아파트']

###### 지역별(1)
['서울']

###### 지역별(2)
['강북권역' '강남권역']

###### 지역별(3)
['도심권' '동북권' '서북권' '서남권' '동남권']

###### 지역별(4)
['종로' '중' '용산' '성동' '광진' '동대문' '중랑' '성북' '강북' '도봉' '노원' '은평' '서대문' '마포'
 '양천' '강서' '구로' '금천' '영등포' '동작' '관악' '서초' '강남' '송파' '강동']

###### 시점
['2018-01' '2018-02' '2018-03' '2018-04' '2018-05' '2018-06' '2018-07'
 '2018-08' '2018-09' '2018-10' '2018-11' '2018-12' '2019-01' '2019-02'
 '2019-03' '2019-04' '2019-05' '2019-06' '2019-07' '2019-08' '2019-09'
 '2019-10' '2019-11' '2019-12' '2020-01' '2020-02' '2020-03' '2020-04'
 '2020-05' '2020-06' '2020-07' '2020-08' '2020-09' '2020-10' '2020-11'
 '2020-12' '2021-01' '2021-02' '2021-03' '2021-04' '2021-05' '2021-06'
 '2021-07' '2021-08' '2021-09' '2021-10' '2021-11' '2021-12' '2022-01'
 '2022-02' '2022-03' '2022-04' '2022-05' '2022-06' '2022-07' '2022-08'
 '2022-09' '2022-10' '2022-11' '2022-12']
(1500, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 5 col

### 전월세전환율

In [25]:
rent_conversion_rt_apt = pd.read_excel(args.default_path + '지역별_전월세전환율.xlsx')
chk_unique(rent_conversion_rt_apt)
print(rent_conversion_rt_apt.head())

# 필요없는 컬럼 제거, 컬럼명 변경
rent_conversion_rt_apt.drop(['지역별(1)',	'지역별(2)','지역별(3)','주택유형별(1)'],axis =1 , inplace =True)
rent_conversion_rt_apt.rename(columns = {'지역별(4)': '자치구','데이터':'전월세전환율'},inplace = True)

chg_date(rent_conversion_rt_apt,'시점','%Y-%m')

# 자치구 전처리
rent_conversion_rt_apt['자치구'] = rent_conversion_rt_apt['자치구'] + '구'

# '월' 열 추가
rent_conversion_rt_apt['연도'] = rent_conversion_rt_apt['시점'].dt.year
rent_conversion_rt_apt['월'] = rent_conversion_rt_apt['시점'].dt.month

chk_df(rent_conversion_rt_apt) # 데이터 확인


###### 주택유형별(1)
['아파트']

###### 지역별(1)
['서울']

###### 지역별(2)
['강북권역' '강남권역']

###### 지역별(3)
['도심권' '동북권' '서북권' '서남권' '동남권']

###### 지역별(4)
['종로' '중' '용산' '성동' '광진' '동대문' '중랑' '성북' '강북' '도봉' '노원' '은평' '서대문' '마포'
 '양천' '강서' '구로' '금천' '영등포' '동작' '관악' '서초' '강남' '송파' '강동']

###### 시점
['2018-01' '2018-02' '2018-03' '2018-04' '2018-05' '2018-06' '2018-07'
 '2018-08' '2018-09' '2018-10' '2018-11' '2018-12' '2019-01' '2019-02'
 '2019-03' '2019-04' '2019-05' '2019-06' '2019-07' '2019-08' '2019-09'
 '2019-10' '2019-11' '2019-12' '2020-01' '2020-02' '2020-03' '2020-04'
 '2020-05' '2020-06' '2020-07' '2020-08' '2020-09' '2020-10' '2020-11'
 '2020-12' '2021-01' '2021-02' '2021-03' '2021-04' '2021-05' '2021-06'
 '2021-07' '2021-08' '2021-09' '2021-10' '2021-11' '2021-12' '2022-01'
 '2022-02' '2022-03' '2022-04' '2022-05' '2022-06' '2022-07' '2022-08'
 '2022-09' '2022-10' '2022-11' '2022-12']
  주택유형별(1) 지역별(1) 지역별(2) 지역별(3) 지역별(4)       시점  데이터
0      아파트     서울   강북권역    도심권     종로  2018-01  4.1
1  

### 지가지수

In [26]:
LPI_month = pd.read_excel(args.default_path + '지가지수_월별.xlsx')
# LPI_m.head()

#컬럼 재설정 -> 지역, 시점 데이터로
LPI_m = LPI_month.iloc[9:]
LPI_m = LPI_m.rename(columns = LPI_m.iloc[0]).drop(LPI_m.index[0])
LPI_m = LPI_m.reset_index(drop=True)
# LPI_m.head()

#필요한 데이터만 - 서울
local= LPI_m['지 역']
cond = (local == '서울') | (local == '부산')
s , b = LPI_m.index[cond]

#슬라이싱
LPI_m = LPI_m.iloc[s:b]

#지역과 nan컬럼 제거
LPI_m.dropna(how='all',axis =1 ,inplace= True)
LPI_m.drop('지 역',axis = 1,inplace = True)
LPI_m.rename(columns = {LPI_m.columns[0]:'자치구'},inplace = True)

# melt
LPI_m.drop(LPI_m.index[0],inplace = True)
LPI_m = pd.melt(LPI_m,id_vars = '자치구',var_name = '시점',value_name = '지가지수')

# to_datetime
chg_date(LPI_m,'시점','%Y년 %m월')

# '월' 열 추가
LPI_m['연도'] = LPI_m['시점'].dt.year
LPI_m['월'] = LPI_m['시점'].dt.month

# 2018년도 부터
LPI_m = LPI_m[LPI_m['연도'] >= 2018]

chk_df(LPI_m) # 데이터 확인

(1625, 5)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1625 entries, 1500 to 3124
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   자치구     1625 non-null   object        
 1   시점      1625 non-null   datetime64[ns]
 2   지가지수    1625 non-null   object        
 3   연도      1625 non-null   int64         
 4   월       1625 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 76.2+ KB
None
      자치구         시점    지가지수    연도  월
1500  종로구 2018-01-01  80.202  2018  1
1501   중구 2018-01-01  80.863  2018  1
1502  용산구 2018-01-01  76.597  2018  1
1503  성동구 2018-01-01  75.717  2018  1
1504  광진구 2018-01-01   79.98  2018  1


### 실거래가격지수

In [27]:
apt_price_index = pd.read_excel(args.default_path + '지역별_공동주택_실거래가격지수.xlsx' )

# 지역와 데이터 값만 가져오기
PI_apt = apt_price_index.iloc[9:]

# 첫번째 행을 열로
PI_apt = PI_apt.rename(columns = PI_apt.iloc[0]).drop(PI_apt.index[0])
# PI_apt.columns

# melt
PI_apt = pd.melt(PI_apt, id_vars='지 역',var_name = '시점', value_name= '실거래가격지수')
# print(PI_apt.head())

# to_datetime
chg_date(PI_apt,'시점','%Y년 %m월')

# 지역 컬럼 제거
PI_apt.drop('지 역',axis =1, inplace =True)

# '월' 열 추가
PI_apt['연도'] = PI_apt['시점'].dt.year
PI_apt['월'] = PI_apt['시점'].dt.month

# 2018년도 부터
PI_apt = PI_apt[PI_apt['연도'] >= 2018]

chk_df(PI_apt) # 데이터 확인

(64, 4)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 64 entries, 24 to 87
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   시점       64 non-null     datetime64[ns]
 1   실거래가격지수  64 non-null     object        
 2   연도       64 non-null     int64         
 3   월        64 non-null     int64         
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 2.5+ KB
None
           시점 실거래가격지수    연도  월
24 2018-01-01     102  2018  1
25 2018-02-01   103.5  2018  2
26 2018-03-01   104.5  2018  3
27 2018-04-01   105.5  2018  4
28 2018-05-01   105.9  2018  5


### 소비심리지수

In [28]:
CSI = pd.read_csv(args.default_path + '소비심리지수.csv' , encoding ='cp949')
# CSI.head()
# chk_unique(CSI) #고유값 확인

# 시도별(1) 컬럼 제거 / 시도별(2) 컬럼명 변경
CSI.drop('시도별(1)',axis =1,inplace =True)
CSI.rename(columns = {'시도별(2)' : '시도별'} , inplace = True)

#서울시의 데이터만
# print(CSI.시도별.unique())
CSI= CSI.loc[CSI.시도별 == '서울특별시']

# 일자 값을 행으로
CSI = pd.melt(CSI,id_vars = '시도별',var_name='시점', value_name = '소비심리지수')
# CSI.info()

# 일자 값 datetime으로 변경
chg_date(CSI,'시점','%Y.%m')

# 시도별 컬럼 제거
CSI.drop('시도별',axis =1 , inplace=True)

# '월' 열 추가
CSI['연도'] = CSI['시점'].dt.year
CSI['월'] = CSI['시점'].dt.month

# 2018년도 부터
CSI = CSI[CSI['연도'] >= 2018]

chk_df(CSI) # 데이터 확인

(64, 4)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 64 entries, 24 to 87
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   시점      64 non-null     datetime64[ns]
 1   소비심리지수  64 non-null     object        
 2   연도      64 non-null     int64         
 3   월       64 non-null     int64         
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 2.5+ KB
None
           시점 소비심리지수    연도  월
24 2018-01-01  126.8  2018  1
25 2018-02-01  126.3  2018  2
26 2018-03-01  116.3  2018  3
27 2018-04-01  108.3  2018  4
28 2018-05-01  109.8  2018  5


## 4) 주거 환경요인

### 상권(편의점, 대형마트, 백화점, 음식점, 카페(프렌차이즈)), 병원 수

In [ ]:
#shop_conv.to_csv('편의점.csv', index=False, encoding='cp949')
shop_convenience=pd.read_csv(args.default_path + '편의점.csv', encoding='cp949')
shop2=pd.read_csv(args.default_path + '대규모점포.csv', encoding='cp949')

In [ ]:
shop=pd.read_csv(args.default_path + '소상공인시장진흥공단_상가(상권)정보_서울_202303.csv')

#필요없는 컬럼 drop

shop.drop(columns=['상가업소번호','상권업종대분류코드','상권업종중분류코드',
                  '상권업종소분류코드','표준산업분류코드','시도코드','시군구코드',
                  '지번코드','대지구분코드','대지구분명',
                   '지번본번지','지번부번지','도로명코드','건물관리번호','구우편번호',
                  '동정보', '층정보','호정보','신우편번호','지번주소','건물본번지','건물부번지'], inplace=True)

# 필요없는 행 삭제 위함
words_to_exclude = ['이용·미용', '섬유·의복·신발 소매' ,'부동산 서비스', '주점', '본사·경영 컨설팅',
                    '광고','기타 교육', '전문 디자인', '기술 서비스', '유원지·오락', '식료품 소매', '법무관련',
                   '스포츠 서비스', '가전·통신 소매', '회계·세무', '사진 촬영', '자동차 수리·세차',
                    '여행사·보조','오락용품 소매','세탁','인쇄·제품제작','청소·방제', '기타 사업 서비스',
                   '교육 지원','일반 숙박', '운송장비 대여','욕탕·신체관리','컴퓨터 수리','시계·귀금속 소매',
                   '중고 상품 소매','애완동물·용품 소매','기타 전문 과학','동남아시아', '기타 가정용품 수리',
                   '가정용품 대여', '고용 알선', '장식품 소매', '일반 교육','식물 소매','기타 숙박',
                   '가구 소매', '가전제품 수리','도서관·사적지', '담배 소매', '시장 조사', '시설관리',
                    '모터사이클 소매','철물·건설자재 소매', '안경·정밀기기 소매', '자동차 부품 소매',
                    '모터사이클 수리', '연료 소매','산업용품 대여', '사무 지원', '통신장비 수리','수의',
                    '조경·유지', '장례식장 ', '기타 개인', '기타 상품 소매', '기타 생활용품 소매','기타 보건',]

for word in words_to_exclude:
    shop.drop(shop[shop['상권업종중분류명'].str.contains(word)].index, inplace=True)

words_to_exclude = ['화장품 소매업','떡/한과','구내식당','의료기기 소매업','그 외 기타 종합 소매업','우유 소매업']

for word in words_to_exclude:
    shop.drop(shop[shop['상권업종소분류명'].str.contains(word)].index, inplace=True)

dele = ['왕가탕후루', '판다탕후루', '스윗쌀롱', '올때붕어빵','과일연구소','묘붕','과일한컵','규리네과일가게','달쥬과일상점잠실2',
        '달빛과일','과일메이드','별과일샵','과일병원','과일조이','달쥬과일상점홍대11호','과일에반하다프루타','달쥬과일상점종로2',
        '규리네과일가게마포','달빛과일강서점','과일냠냠','과일연구소','과일담은요거트맛집yomat','코하프루트&요거트','달콤프루트',
        '싱싱프루츠','간식창고신방화','슬기로운간식생활','일월만두아이스크림','일월푸드만두아이스크림','모찌네콩떡빙수',
        '달콤빙고','샤빙','빙볼','크레이지빙수건대','순백빙수','신빙고','마망드빙떡','북해빙수','케밥빙',
        '강남후르츠','국민후르츠','샐러드&후르츠','스마일명품찹쌀꽈배기','몽키꽈배기','명품찹쌀꽈배기','꿀꽈배기','꽈배기상회',
        '스마일명품꽈배기','서울은마꽈배기','성수동꽈배기로고스타','경성찹쌀꽈배기','경성꽈배기','김부자꽈배기','이가꽈배기',
        '경성꽈배기송파역점','스마일명품찹쌀꽈배기','아리꿀꿀찹쌀꽈배기','허부자꽈배기','스마일명품찹쌀꽈배기',
        '꽈대리옥수수꽈배기','황금찹쌀꽈배기','복코끼리꽈배기','스마일명품찹쌀꽈배기','미미고찹쌀꽈배기길동역점','맛고을꽈배기',
        '신사꽈배기','황금찹쌀꽈배기','경성꽈배기상일동역점','찹쌀미친꽈배기','그의사랑!홍대꽈배기','미친꽈배기',
        '못난이찹쌀꽈배기','은마꽈배기','골목길찹쌀꽈배기','경성찹살꽈배기','경성찹쌀꽈배기우','미미고찹쌀꽈배기고덕베네루체점',
        '황제찹쌀꽈배기','더꽈배기','경성꽈배기','떡호랑케이크샵','양자호떡','총각네호떡','이정순호떡','서울호떡','삼맛호오떡',
        '미아리호떡','목동찹쌀호떡','호떡집','점순이호떡앤핫도그','점순이호떡','떡깨비','서울호떡사당','떡세스','꾸러기밤호떡',
        '양자호떡','하니행운호떡','점순이호떡','서울호떡신촌','아빠호떡','덤프PC카페','정선아리랑찐빵','다우가왕만두찐빵',
        '정수네옛날왕만두찐빵','구구점빵','황금잉어빵','광장총각네붕어빵','미호붕어빵연남점','안흥빵집','에스지푸드우리쌀빵',
        '조선제일빵','황금똥빵','유성찐빵','복가네명품만두찐빵','다우가왕만두찐빵','그릴브레드','모노브레드','브레드홍',
        '마법사꽈베기도너츠','허니찹쌀꽈베기','토리만쥬용산역점','만쥬킹홍대입구역','마이수플레','미소당베이커리회기점',
        '호두톡톡','선녀호두파이','호두과자','복호두강남','호두앤미수락산점','호두앤미송파','복호두DMC점','호두당',
        '영진이노텍호두붐왕십리','복호두','호두킹','호두앤미', '이가네꽈배기','못난이꽈배기암사',
        '탭하트케이크','떡가게종춘익선','빵','니캉내캉꽈배기','피오니케이크', '헤버나데제과점','바이재재빵굽는자매',
       '코코호도','포유플라워케이크','탭하트케이크','꾸꾸케이크','밍스케이크','피오니케이크','아라주라케이크','이든케이크','도오케이크',
        '설희케이크','알리케이크','케이크바이라미','000케이크바','온오프케이크','겸재케이크','아토치즈케이크','윙크케이크','팬토리케이크',
        '벨케이크','바림케이크','오늘엔케이크','목하케이크','플랑케이크','멜로디케이크','열두달케이크','잇마이케이크','케이크소원',
        '라온앙금플라워케이크','오디너리케이크','유니아케이크','21케이크','루모스케이크','베니케이크','야미케이크','키라키라치즈케이크',
        '키키네케이크','위시어폰어케이크','이아컵케이크','마따미스케이크','뉴띵케이크','오브케이크','칠요일의케이크','고마워케이크',
        '데이지케이크','유니네케이크','구르미케이크','요미앙케이크','딜라이트디케이크','리나케이크','강남고망고케이크''모드니케이크',
        '앙또와케이크','꾸움케이크''해피해피케이크','치크치크케이크','루나리스케이크''에스텔플라워케이크','퐁실케이크','예쁘당케이크',
        '아틀리에401케이크','단꽃케이크','엘모리아케이크','리레케이크','빙키케이크','츄카츄카케이크','보통의케이크','히포케이크',
        '두두케이크','조은날케이크','원데잇케이크','키라키라케이크','하라케이크','케이크','이크케이크','리블리케이크','오케이크',
        '아코케이크','기글케이크','시루아케이크','닷트케이크','윤별케이크','라라슈케이크','슈기케이크','보기조은케이크','기분케이크',
        '마롱케이크','리슨케이크','본비케이크','우노케이크','빅조이케이크','13월의케이크','아뜨레케이크','로멘토케이크','오케이크',
        '연케이크','스코프케이크하우스','케이크를부탁해','아로케이크','케이크풀','분홍케이크샵','케이크예스플리즈','블루벨케이크',
        '아워모멘트케이크','리프케이크','딩가케이크','굿나이스파인케이크샵','커스버트케이크','와이즈케이크와is케이크','케이크라보',
        '꽃케이크꼬크','시루케이크','아니타케이크앤스위츠','딜릴리케이크','마들케이크','이백이케이크','니니케이크','버니케이크',
        '위켄드케이크','해이케이크샵','오렌지케이크메이커','케이크케이크','구슬케이크','우리플라워케이크','재재케이크','러시안케이크',
        '아이라이크케이크','플로스원케이크','아네모네시케이크','세이케이크','캔버스케이크컴퍼니','구뜨케이크','김그린케이크',
        '담다꽃케이크','꼼스케이크','아이비케이크','케이랑케이크','알레나케이크','오케이크','꼬모케이크','서래케이크','빌런치팬케이크',
        '헤이미쉬케이크','너와나의케이크','선자케이크','케이크츄','모든것은당근케이크에서시작됐다','러시아케이크','맑은날케이크',
        '리썸케이크','어니스트팬케이크','케이크칼미아','루시케이크','달봉케이크','오드투케이크','집사의케이크','디케이크앤디저트',
        '케이크더조아','디니케이크','크림온케이크','유일케이크','보니케이크','베라케이크','노랑설탕플라워케이크','비비디케이크',
        '하찌케이크','아델케이크','설레임케이크','유니케이크','원더시티!뉴욕컵케이크','케이크만들기단하나','케이크를부탁해',
        '달코롬케이크','브라운잇케이크','케이크의계절','위싱케이크','케이크를부탁해','포유케이크','2NaRy플라워케이크','딜로케이크',
        '구르미케이크','먼슬리케이크','굿무드케이크','민정씨네케이크','보틀케이크','바나케이크','비비케이크','남다른케이크',
        '양크림케이크','더샌드&케이크','햄니케이크','보보케이크','도잉케이크','포레스트케이크','케이크지','선아케이크','그러케이크샵',
        '모든것은당근케이크에서시작됐다','스윗드로우케이크','이츠어피스오브케이크','케이크앤코','하므케이크','한국파티케이크디자인협회',
        '리츠케이크','로코케이크','오리지널팬케이크하우스가로수길점','케이크아뜰리에','케이크케이크','케이크달달','단하나케이크',
        '폼즈케이크샵','샘케이크','오앤오케이크', '구움케이크샵','릿츠케이크','해브어케이크데이','별난케이크','코코케이크','케이크쥐',
        '언제나케이크','이파슈케이크','언제든지케이크','다락방케이크','제이솔케이크','연두플라워케이크','김미케이크','은하수케이크',
        '영케이크','케이크팝','로에케이크','보라곰케이크','로에케이크','뵈뵈케이크팝','플디케이크샵','서울케이크','코코호도답십리역점',
        '코코호도센트럴시티점','코코호도강남','코코호도은평뉴타운점','코코호도구.신천점','코코호도창동하나로점','코코호도도봉',
        '코코호도쌍문점','코코호도서소문점','코코호도명동성당점','코코호도상도역점','코코호도방화역점','비지티호도단팥빵북아',
        '코코호도목동','코코호도한티역점''코코호도여의도','코코호도수서역점','삼송빵집영등포역사점','금은빵','서울빵집','빵굽는쉐프',
        '자양빵공장','빵','바이재재빵굽는자매','푸하하크림빵','북촌명인단팥빵','이재원제빵소','배빵빵','대치동제빵소''구름빵제과점',
        '파란고양이와단팥빵','반달제빵소','계란빵사세요','빵야빵야','곰세마리수제식빵','빵내솔','빵사부식빵공방동덕여대점',
        '팥빵애빠지다','고덕빵집','빵데이','즉석빵집','김도봉제빵소','빵집&꽈배기','인정식빵','호호빵집','빵이좋은사람들',
        '언니의식빵가게흑석점','구름빵베이커리','쌍빵데레','식빵이야기신풍래미안점','그레인제빵소','빵굽는이야기밀담','빵앤뺑',
        '자연식빵','식빵','한나식빵신대방삼거리역점','도제식빵','빵쌤연희점','식빵이야기둔촌역점','빵에반하다','빵드몽','빵굽는요셉',
        '빵팔구','아빵','한나식빵브레드카페','울할매빵집','달콤제빵소','수제빵미소빵긋','빵굽는아저씨','빵을위한시간','노원제빵소',
        '라보뒤빵','빵카페미니신방화점','백호빵집','더르빵','식빵이야기','바로굼빵터','옥순빵집','빵데이','빵이가','송파제빵소',
        '석내빵집','완두찐빵','오늘의빵','담뿍빵집','종로명인단팥빵','북촌유기농밀명인단팥빵','한국요유빵마포지점','수제빵수베이커리',
        '우리동네식빵','동화속빵과자점','규동빵집','북촌유기농밀명인단팥빵','빵명가','그랑프리빵','빵카페','맘식빵','양씨네제빵소',
        '곰돌이빵집','오빵이씨','우리쌀빵','식빵이주인공','마포빵집','모퉁이빵집','묘한빵집','에스엠에이치삼송빵집','아주느린빵',
        '빵굽는아빠','빵파라다이스','빵집오늘','빵명장외대점','빵쌀롱','인수빵집','바푸리수제식빵','유가네빵집','태순당즉석빵','삼송빵집',
        '삼송빵집코엑스','한나식빵','인유단단팥빵','빵장수단팥빵','우리쌀빵','빵쌤','경성제빵소','냥빵','빵이가득한집','언니의식빵가게',
        '정수제빵소','빵빵카롱','경주황남빵롯데월드몰','빵어니스타','달콤제빵소','빵.굽.남','가드니아여주제빵소','르빵','그대를위한식빵',
        '빵카페1','언덕마을식빵','갓구운식빵,오늘','오르리단팥빵','바로굼빵터','꿀바른제빵소','빵의향현','빵이좋아베이커리','캉파뇽제빵소',
        '우리쌀빵','빵굽는쉐프','웰빙쌀빵','빵이가득한집','두빵','빵장수단팥빵강남.','빵공방','국민빵집','한나식빵','빵굼터명일동점',
        '안가네대추빵','식빵이야기','빵쌤마포','빵빵','버블호떡찰붕어빵','달맞이빵베이커리','에스엠에이치삼송빵집잠실','브라운필빵공장',
        '식빵이야기','빵그레','사랑빵손님','빵장수단팥빵','빵장수단팥빵','찰리의빵공장','빵가마베이커리','빵과함께','빵또와베이커리',
        '우리동네식빵해적단','빵사랑','나미야빵집','빵긋한하루','빵이가득한집','빵터질라','떡빵앗간','일호빵','공공7빵','르빵프레',
        '빵쌤경희대점','우리쌀빵','빵쌤','식빵이야기','따뜻한식빵','아리랑빵집','농부아버지빵굽는딸','빵하우스','벨기에빵집',
        '푸하하크림빵','건강한빵','빵도넛','두란노제빵소','빵굽는마을','얌얌브레드X을지점빵','베들레헴빵','비지티호두단팥빵(불광)',
        '빵언니의식빵공장','브레드팩토리빵야','또바기빵','체부동다락빵','우리동네식빵','빵앗간토끼','신빵굼터','빵글','베이커리이씨네빵집',
        '빵데이','북촌유기농밀명인단팥빵','빵쌤','밀내음단팥빵','루이제빵소','식빵굽는남자','단석가찰보리빵','우리쌀빵&웰빙쌀빵','빵장이',
        '소화연빵','소반술빵','선유빵집','싱글벙글빵도너츠','빵하나의즐거움','이남자의빵','유심빵집','브니엘빵집','밀밭제빵소목동',
        '홍순왕만두찐빵','덕순이빵집','연희동국화빵','지사부왕만두찐빵','느루찐빵','거북이빵집','빵빵한카스테라','베이커리빵씨',
        '바로굼빵터','빵&빵','쁘띠빵빵','빵굽는쉐프의꿈','빵마을과자점','미소식빵','빵나무','우리동네식빵','효모빵집','빵굽는동네',
        '보리빵앗간','빵/도넛','구름빵카페','빵굽는아빠','티나의식빵','바로굼빵터','빵이야기','독일빵집','빵굽는사랑방','빵사이에낀과일',
        '바로굽는빵집','비지티호도단팥빵북아','빵굽는작은언덕','빵굼터','오르리단팥빵','언니의식빵가게','빵꽃빵꽃','빵집플러스커피',
        '빵굼터착한빵집','옛날빵집','식빵210','에스지푸드웰빙쌀빵가양역','빵집아저씨','삐에스몽테제빵소','빵이가득한집','빵빵데이',
        '난곡제빵소','빵굽는나라','수유제빵소','빵굼터착한빵집','황둔찐빵','빵빵한식빵가게','홍가네빵굼터','겹겹이빵집','신림빵할인점',
        '빵이가득한집','까미노빵집','갓구운빵집','민이네식빵','빵장수단팥빵성','안가네대추빵','소보리제빵소','빵찌하우스','우리동네식빵',
        '삼봉식빵','비지티호두단팥빵','상도제빵소','델꾸와식빵','서병래제빵소','우체국옆빵집','골목빵집','팡야빵야','빵야베이커리',
        '르빵LEPA','빵굼터','빵에빠지다','빵장수단팥빵우','빵사랑베이커리','소문난빵집','독일빵집','그랑프리빵','이상미통밀빵',
        '청춘식빵','이가네제빵소','경성빵공장올림픽공원점','빵터진유명인','우리쌀빵','빵이가득한집','빵굽는작은마을','만나빵집',
        '빵굽는드림하우스','블럭제빵소','빵이요베이커리','제빵왕','유명한빵집','신수동,작은빵집','신빵굼터','바로굼빵터','행복한빵세상',
        '맛있는빵집','아리랑빵집','빵굽는마을','빵굽는이야기','빵사랑베이커리','빵이가득한집','빵이당','클랑도쿄빵야','빵쌤',
        '빵이야기','명가인빵','빵굼터착한빵집','빵나라','빵굼터','빵/도넛','빵이가득한마을','르빵','빵마을','잠실제빵소','소보로빵집',
        '방이빵집','숙이네찐빵','빵굼터','우리동네식빵','빵장수요셉','빵굽는마을','은행빵집','빵굽는사람들','행복한빵집','오늘제빵소',
        '카페사랑빵','즉석빵','우리쌀빵','더불어함께숲북한산큰숲,제빵소','킷토빵','빵드밀','쌍문빵곳간','한나식빵서대문역점','배빵빵',
        '서울연인단팥빵','빵이야기','빵미제빵소','그레인제빵소','몰빵중화역지점','빵굽는카페케이','오늘의빵','빵이가득한집','바로굼빵터',
        '한나식빵우면점','교대쌀빵','명성제빵소','빵굽는밀밭제빵소','악어빵집','건빵','앞으로의빵집','맛있는빵집','대한제빵소','빵심',
        '한나식빵황학동롯데','빵이가','빵의시간','행운빵집','복희빵집','밀밭제빵소서초','연신내빵집','비지티호두단팥빵','찰찰보리식빵',
        '대치동제빵소베이커리까페','빵내음솔솔','빵굼터영등포구청역점','빵카페','호두단팥빵','이가네꽈배기','못난이꽈배기암사',
        '니캉내캉꽈배기','단골많은꽈배기','최고의꽈배기','꽈배기마법사','꽈배기의달인','빵집&꽈배기','스마일찹쌀꽈배기','요즘꽈배기',
        '콩꼬물꽈배기항동점','원더풀꽈배기','황제꽈배기','스마일북한산명품꽈배기','안녕,꽈배기','왕십리꽈배기로고스타','남한산성꽈배기',
        '을지로스마일꽈배기','콩꼬물꽈배기','황제꽈배기','가마솥꽈배기삼미','스마일꽈배기','광장시장찹쌀꽈배기','명품찹쌀핫도그&꽈배기',
        '근덕이삼척꽈배기','압구정꽈배기','못난이꽈배기','미미고찹쌀꽈배기광나루점','스마일찹쌀꽈배기','말죽거리꽈배기','스타도로시꽈배기',
        '못난이꽈배기','에스비못난이꽈배기','스마일꽈배기곰달래점','콩꼬물꽈배기망원역점','올레찹쌀꽈배기','못난이꽈배기강북북부',
        '별빛꽈배기','꽈배기당','마법사꽈배기','진미당명품꽈배기','쉐프윤꽈배기고로케','신대방대박찹쌀꽈배기','꽈배기스토리',
        '길음동스마일꽈배기','스마일꽈배기','이가네꽈배기','쌍문동꽈배기by꽈백최선생','화승꽈배기','스마일찹쌀꽈배기',
        '못난이꽈배기목5동점','콩꼬물꽈배기','김호연꽈배기도너츠','스마일찹쌀꽈배기','미친찹쌀꽈배기','꿀복이꽈배기',
        '쉐프윤찹쌀꽈배기고로케','미미고찹쌀꽈배기','경리단꽈배기','스마일찹쌀꽈배기','쉐프윤찹쌀꽈배기고로케','꽈배기팩토리2',
        '바른꽈배기','황금꽈배기','신월동꽈배기','청담꽈배기','착한꽈배기','손에안묻는찹쌀꽈배기','뚝섬찹쌀꽈배기','스마일꽈배기',
        '숙꽈배기','꽈배기스토리','새참꽈배기','츄잉떡','떡가게종춘익선','떡그리다','호떡당','목동망개떡','김진호호떡','북촌호떡',
        '씨앗호떡','훈훈호떡','꽃떡공방','그림의떡','여미향망개떡','마이따호떡','버블호떡찰붕어빵','보리떡과물고기','떡빵앗간',
        '지니떡방','이낙근찹쌀떡베이커리','철철호떡','남대문야채호떡','호떡당도심휴게소','케익샵떡송송','이낙근찹쌀떡베이커리카페',
        '떡까페시혁','하루떡','영주당쌀강정오방떡','사랑의떡마을','설송원망개떡','행복떡방&Cafe','이씨네호떡','삼청동호떡',
        '풍년쑥개떡','송이네호떡'

]
for word in dele:
    shop.drop(shop[shop['상호명'].str.contains(word)].index, inplace=True)

# 여러 개의 값 리스트
cafe = ['카페도비','쌍문동카페','오빠생각타코야끼&캔카페','푸드카페','양키스카페','정자매카페','카페라로','별난만두카페',
        '찬상반찬카페','패션카페더플러스민트','라몽드카페','카페원더','카페코드','슈트카페테리아','카페크라우드',
        '더블하이트갤러리카페','키즈카페폭스맘','카페뤽상부르','더드림카페드림메디힐','카페양화진','군산카페','타코야끼카페',
        '반지만들기카페건대지점','마이쥬카페','카페후','카페453','나이트카페','콩카페','카페마리','해례커피','봉명동내커피',
        '비비드커피','호구커피','쏙만두&커피','모멘트커피','커피당','이모티프커피바','최군커피','더맛있는과일&1L커피',
        '셀렉티드커피','인스텐드커피','맘씨커피','새시로커피랩','스트릿커피','메가엠지씨커피도선','메가엠지씨커피신용산역점',
        '맘fun한cafe','caffe&츄러스','까페베르떼','비아찌오', '카페오네이', '로코카페', '봄브레드', '크로플덕',
        '크로플덕오리아가씨', '주주크로플','설빙양재역점']

condition = shop['상호명'].isin(cafe)

shop.loc[condition, '상권업종소분류명'] = '카페'
shop.loc[condition, '상권업종대분류명'] = '음식'

word=['베이커리', '제과점', '마카롱', '브레드', '빵']
condition = shop['상호명'].isin(word)

for word in dele:
    shop.drop(shop[shop['상호명'].str.contains(word)].index, inplace=True)

def filter_dataframe(shop, condition1, condition2):
    shop_cafe = shop[(shop['상권업종소분류명'] == condition1) | (shop['상권업종소분류명'] == condition2)]
    return shop_cafe

# 음식점 df
conditions = ['곱창 전골/구이', '백반/한정식', '김밥/만두/분식', '치킨',
              '피자', '중국집', '돼지고기 구이/찜', '족발/보쌈', '국수/칼국수',
             '횟집','마라탕/훠궈', '국/탕/찌개류','경양식', '토스트/샌드위치/샐러드',
             '일식 회/초밥','냉면/밀면', '복 요리 전문','닭/오리고기 구이/찜',
              '그 외 기타 간이 음식점', '버거','기타 한식 음식점','해산물 구이/찜',
              '일식 면 요리','파스타/스테이크','전/부침개','소고기 구이/찜','뷔페',
              '기타 서양식 음식점','일식 카레/돈가스/덮밥','패밀리레스토랑','기타 일식 음식점']
shop_restaurant = shop[shop['상권업종소분류명'].isin(conditions)]

shop_restaurant_count = shop_restaurant['법정동명'].value_counts().reset_index()
shop_restaurant_count.columns = ['법정동명', '음식점 수']

# 카페 df
shop_cafe = filter_dataframe(shop, '빵/도넛', '카페')
shop_cafe_count = shop_cafe['법정동명'].value_counts().reset_index()
shop_cafe_count.columns = ['법정동명', '카페 수']

# 병원 df
shop_hospital=shop[shop.상권업종대분류명=='보건의료']
shop_hospital_count = shop_hospital['법정동명'].value_counts().reset_index()
shop_hospital_count.columns = ['법정동명', '병원 수']

# 편의점 df
words_to_remove = ['의약·화장품 소매', '음료 소매']

# 단어가 포함된 행 제거
for word in words_to_remove:
    shop = shop[~shop['상권업종중분류명'].str.contains(word)]

keywords = ['CU', 'cu','씨유', '미니스톱', '지에스25', 'GS25', '세븐일레븐','세븐' '쎄븐일레븐', '이마트24', 'EMART24']

shop_conv=shop[shop['상호명'].str.contains('|'.join(keywords))]

shop_convenience_count = shop_convenience['법정동명'].value_counts().reset_index()
shop_convenience_count.columns = ['법정동명', '편의점 수']

shop_convenience_count = shop_convenience['법정동명'].value_counts().reset_index()
shop_convenience_count.columns = ['법정동명', '편의점 수']

# 마트(대형마트, 백화점 상권)
shop2=shop2.fillna(' ')
shop2=shop2[shop2.소재지전체주소.str.contains('서울')]

words_to_remove = ['슈퍼', '수퍼', 'GS', '에브리데이', '시장', '상가', '프라자', '마켓', '리테일', '쇼핑']

# 단어가 포함된 행 제거
for word in words_to_remove:
    shop2 = shop2[~shop2['사업장명'].str.contains(word)]

##### 여기까지 코드로 하고 나머지 csv로 추출해서 계속 엑셀로 확인하면서 진행함. 아마 중간에 하면서 엑셀에서 손봤을 수 있어서 데이터가 달라질수 있어요. merge할때 대규모점포2.csv로 불러와서 여기부터 코드 돌리는게 나을것같아요 #####

shop2['도로명전체주소2']=shop2['도로명전체주소'].apply(lambda x: x.split('('))
shop2['법정동명']=shop2['도로명전체주소2'].str[1]
shop2['법정동명']=shop2['법정동명'].astype(str).apply(lambda x: x[:-1])
shop2.drop(columns=['도로명전체주소2', '점포구분명','데이터갱신구분', '최종수정시점','도로명우편번호','소재지전체주소'], inplace=True)

# 백화점 df
shop_bigmart=shop2[shop2.업태구분명=='대형마트']
shop_department=shop2[shop2.업태구분명=='백화점']

shop_department_count = shop_department['법정동명'].value_counts().reset_index()
shop_department_count.columns = ['법정동명', '백화점 수']

# 대형마트 df
shop_bigmart_count = shop_bigmart['법정동명'].value_counts().reset_index()
shop_bigmart_count.columns = ['법정동명', '대형마트 수']

# 상권 데이터 merge

# 여러 개의 데이터프레임을 병합
dfs = [shop_restaurant_count,shop_cafe_count,shop_convenience_count,shop_hospital_count,shop_bigmart_count,shop_department_count]
comm_district = dfs[0]  # 첫 번째 데이터프레임을 기준으로 초기화

for df in dfs[1:]:
    df['법정동명'] = df['법정동명'].astype(str)
    comm_district = pd.merge(comm_district, df, on=['법정동명'], how='left')

comm_district=comm_district.fillna(0)

comm_district['카페 수']=comm_district['카페 수'].astype('int')
comm_district['편의점 수']=comm_district['편의점 수'].astype('int')
comm_district['병원 수']=comm_district['병원 수'].astype('int')
comm_district['대형마트 수']=comm_district['대형마트 수'].astype('int')
comm_district['백화점 수']=comm_district['백화점 수'].astype('int')

chk_df(comm_district)

In [29]:
#최종 상권 데이터 - 법정동 데이터 추가한 뒤 최종
comm_district = pd.read_csv(args.default_path + '최종_상권_데이터_자치구_법정동.csv', encoding='cp949')

### 5대 범죄 발생 현황

In [30]:
crime = pd.read_csv(args.default_path + '5대_범죄_발생현황.csv')

# 불필요한 데이터 제거(합계, 발생)
crime.drop(crime.index[1],inplace = True) #합계
crime.drop(crime.index[2],inplace = True) #발생

# 시점 데이터 컬럼
crime = crime.T.reset_index(drop=True)
crime = crime.rename(columns=crime.iloc[0]).drop(crime.index[0])

# 컬럼명 변경
crime.columns=['자치구','5대범죄','2018',	'2019',	'2020',	'2021',	'2022']

# 자치구별 범죄 소계 데이터
cond = crime['5대범죄']== '소계'
crime = crime[cond].iloc[1:]

# melt
crime = pd.melt(crime,id_vars=['자치구','5대범죄'],var_name='연도',value_name='5대범죄(총)')
crime.drop('5대범죄',axis=1, inplace = True)

chk_df(crime)

(125, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   자치구      125 non-null    object
 1   연도       125 non-null    object
 2   5대범죄(총)  125 non-null    object
dtypes: object(3)
memory usage: 3.1+ KB
None
   자치구    연도 5대범죄(총)
0  종로구  2018    3690
1   중구  2018    4030
2  용산구  2018    3411
3  성동구  2018    2457
4  광진구  2018    3915


## 5) 일자리 인프라 요인

### 경제활동인구

In [126]:
economy_act = pd.read_csv(args.default_path+'행정구역_시도_별_경제활동인구_20230710133121.csv',  encoding='cp949')

In [127]:
economy_act

,시도별,시점,경제활동인구 (천명)
0,서울특별시,2018,5335
1,서울특별시,2019,5319
2,서울특별시,2020,5293
3,서울특별시,2021,5312
4,서울특별시,2022,5295


### 시군구별 고용보험 피보험자 수

In [130]:
employment_insurance = pd.read_csv(args.default_path+'월별_피보험자현황.csv',  encoding='utf-8')

employment_insurance.rename(columns = {'(사업장)시군구' : '시군구', '피보험자수(전체)' : '피보험자수', '마감년도' : '년도'} , inplace = True)

#split 메서드로 열 이름을 분리
employment_insurance['연도'] = employment_insurance['년도'].apply(lambda x: re.findall(r'\d{4}', x)[0])

#split 메서드로 열 이름을 분리
v_split = employment_insurance.시군구.str.split(' ')

si = v_split.str.get(0)
gu = v_split.str.get(1)
employment_insurance['시'], employment_insurance['구'] = (si, gu)

employment_insurance['피보험자수'] = employment_insurance.피보험자수.str.replace(',', '').astype('int64')

employment_insurance = employment_insurance[['시군구', '연도', '시', '구', '피보험자수']]

chk_df(employment_insurance)

(125, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시군구     125 non-null    object
 1   연도      125 non-null    object
 2   시       125 non-null    object
 3   구       125 non-null    object
 4   피보험자수   125 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 5.0+ KB
None
         시군구    연도      시    구    피보험자수
0  서울특별시 종로구  2018  서울특별시  종로구  2942079
1  서울특별시 종로구  2019  서울특별시  종로구  2890067
2  서울특별시 종로구  2020  서울특별시  종로구  2890661
3  서울특별시 종로구  2021  서울특별시  종로구  2873020
4  서울특별시 종로구  2022  서울특별시  종로구  3017588


### 지역별 사업체수/사업 유형

In [ ]:
business = pd.read_csv(args.default_path+'2021사업체조사결과.csv')
business_bigcode = pd.read_csv(args.default_path+'산업대분류코드.csv')
business_smallcode = pd.read_csv(args.default_path+'산업세세분류코드.csv')
business_adm = pd.read_csv(args.default_path+'사업체_행정동정보.csv')

# 사업체 조사
## 필요컬럼 추출
business_1 = business[['SURV_BASE_YEAR', 'AD_CD', 'FND_YEAR', 'ORG_FORM_CD', 'INDST_LCLS_CD', 'MBZ_INDST_DIV_CD']]

## 컬럼명 변경
business_1.rename(columns={'SURV_BASE_YEAR':'조사년도', 'AD_CD':'시군구코드', 'FND_YEAR':'창업년도', 'ORG_FORM_CD':'조직형태', 'INDST_LCLS_CD':'업종대분류', 'MBZ_INDST_DIV_CD':'업종소분류'}, inplace=True)

## 시군구 코드에서 앞의 숫자 11로 서울 지역이 포함된 데이터만 남기고 나머지는 제거
business_2 = business_1[business_1['시군구코드'] < 2000000]

## 조직형태 내용 바꾸기
typinfo = {1:'개인사업체', 2:'회사법인', 3:'회사이외법인', 4:'비법인단체', 5:'국가/지방자치단체'}
business_2['조직형태'].replace(typinfo, inplace=True)

# 산업대분류코드, 산업세세분류코드, 사업체_행정동정보 컬럼명 변경 및 통일
## 산업대분류 코드
business_bigcode['항목명'] = business_bigcode['항목명'].apply(lambda x: re.sub(r'\([^()]+\)', '', x))

## 산업세세분류 코드
### 코드에 포함된 온점'.'제거
business_smallcode['코드'] = business_smallcode['코드'].astype('str')
business_smallcode['코드'] = business_smallcode['코드'].str.replace(pat=r'[^\w]', repl=r'', regex=True)
### 'nan' 으로 표시된 행 제거
business_smallcode.drop([557], axis=0, inplace=True)

### 수치형으로 타입변경
business_smallcode['코드'] = business_smallcode['코드'].astype('int64')

## 두개의 데이터프레임 컬럼명 변경
business_bigcode.rename(columns={'코드':'업종대분류', '항목명':'업종대분류 항목'}, inplace=True)
business_smallcode.rename(columns={'코드':'업종소분류', '항목명':'업종소분류 항목'}, inplace=True)
business_adm.rename(columns={'소분류':'시군구코드', '시군구':'구', '읍면동':'동'}, inplace=True)

## 데이터프레임 병합
### merge_1 '업종대분류'를 기준으로 병합
business_merge_1 = pd.merge(business_2, business_bigcode, how='left', on='업종대분류')
### merge_2 '업종소분류'를 기준으로 병합
business_merge_2 = pd.merge(business_merge_1, business_smallcode, how='left', on='업종소분류')
business_merge_3 = pd.merge(business_merge_2, business_adm, how='left', on='시군구코드')
seoul_business = business_merge_3[['시도', '구', '동', '창업년도', '조직형태', '업종대분류 항목', '업종소분류 항목']]

## 결측치 제거
seoul_business = seoul_business.dropna()

## 사업체 수
business_count = seoul_business.groupby(['구', '동']).size().reset_index(name='동별 사업체 수')

# merge를 위한 값 변경
business_count.loc[business_count['동'] == '금호2·3가동', '동'] = '금호2.3가동'
business_count.loc[business_count['동'] == '면목3·8동', '동'] = '면목3.8동'
business_count.loc[business_count['동'] == '상계3·4동', '동'] = '상계3.4동'
business_count.loc[business_count['동'] == '상계6·7동', '동'] = '상계6.7동'
business_count.loc[business_count['동'] == '종로1·2·3·4가동', '동'] = '종로1.2.3.4가동'
business_count.loc[business_count['동'] == '종로5·6가동', '동'] = '종로5.6가동'
business_count.loc[business_count['동'] == '중계2·3동', '동'] = '중계2.3동'

# merge를 위한 값 변경
seoul_business.loc[seoul_business['동'] == '금호2·3가동', '동'] = '금호2.3가동'
seoul_business.loc[seoul_business['동'] == '면목3·8동', '동'] = '면목3.8동'
seoul_business.loc[seoul_business['동'] == '상계3·4동', '동'] = '상계3.4동'
seoul_business.loc[seoul_business['동'] == '상계6·7동', '동'] = '상계6.7동'
seoul_business.loc[seoul_business['동'] == '종로1·2·3·4가동', '동'] = '종로1.2.3.4가동'
seoul_business.loc[seoul_business['동'] == '종로5·6가동', '동'] = '종로5.6가동'
seoul_business.loc[seoul_business['동'] == '중계2·3동', '동'] = '중계2.3동'

# 각 구, 동별 업종대분류 항목 기준 사업체 수
business_big = seoul_business.groupby(['구', '동', '업종대분류 항목']).size().reset_index(name='업종대분류 수')

print('business_big:', len(business_big))

# Convert long-format to wide-format
business_big_wide = business_big.pivot(index=['구', '동'], columns='업종대분류 항목', values='업종대분류 수')
business_big_wide.reset_index(inplace=True)
business_big_wide

In [131]:
### 최종 사업체 현황 - 법정동 대대적 수정 필요했음.
business_big_wide_real_final = pd.read_csv(args.default_path + 'business_modified_bubjeong_df.csv')

## 6) 자연 환경 요인

### 서울시 주요 공원현황

In [33]:
park = pd.read_csv(args.default_path + '서울시 주요 공원현황.csv', encoding = 'cp949')

#split 메서드로 열 이름을 분리
v_split = park.공원주소.str.split(' ')

gu = v_split.str.get(1)
park['구'] = gu

In [34]:
# 네이버 지도 api를 통한 위경도 데이터 추가하기
def reverse_geocoding(lat, lng, client_id, client_secret):
    url = "https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret
    }
    params = {
        "coords": f"{lng},{lat}",
        "sourcecrs": "epsg:4326",
        "orders": "legalcode,admcode,addr,roadaddr",
        "output": "json"
    }
    response = requests.get(url, headers=headers, params=params)
    data = response.json()

    try:
        legalcode_code = data["results"][0]['code']['id']
        legalcode_si = data["results"][0]["region"]["area1"]["name"]
        legalcode_gu = data["results"][0]["region"]["area2"]["name"]
        legalcode_dong = data["results"][0]["region"]["area3"]["name"]
        admcode_code = data["results"][1]['code']['id']
        admcode_si = data["results"][1]["region"]["area1"]["name"]
        admcode_gu = data["results"][1]["region"]["area2"]["name"]
        admcode_dong = data["results"][1]["region"]["area3"]["name"]
        addr_si = data["results"][2]["region"]["area1"]["name"]
        addr_gu = data["results"][2]["region"]["area2"]["name"]
        addr_dong = data["results"][2]["region"]["area3"]["name"]
        addr_bunji1 = data["results"][2]["land"]["number1"]
        addr_bunji2 = data["results"][2]["land"]["number2"]
        roadaddr_si = data["results"][3]["region"]["area1"]["name"]
        roadaddr_gu = data["results"][3]["region"]["area2"]["name"]
        roadaddr_dong = data["results"][3]["region"]["area3"]["name"]
        roadaddr_roadname = data["results"][3]["land"]["name"]
        roadaddr_detail = data["results"][3]["land"]["number1"]

    except (KeyError, IndexError):
        legalcode_code = np.nan
        legalcode_si = np.nan
        legalcode_gu = np.nan
        legalcode_dong = np.nan
        admcode_code = np.nan
        admcode_si = np.nan
        admcode_gu = np.nan
        admcode_dong = np.nan
        addr_si = np.nan
        addr_gu = np.nan
        addr_dong = np.nan
        addr_bunji1 = np.nan
        addr_bunji2 = np.nan
        roadaddr_si = np.nan
        roadaddr_gu = np.nan
        roadaddr_dong = np.nan
        roadaddr_roadname = np.nan
        roadaddr_detail = np.nan

    return legalcode_code, legalcode_si, legalcode_gu, legalcode_dong, admcode_code, admcode_si, admcode_gu, admcode_dong, addr_si, addr_gu, addr_dong, addr_bunji1, addr_bunji2, roadaddr_si, roadaddr_gu, roadaddr_dong, roadaddr_roadname, roadaddr_detail

In [35]:
# # 네이버 API 인증 정보
# client_id = ""
# client_secret = ""

# # 새로운 열을 추가할 리스트
# legalcode_code_list = []
# legalcode_si_list = []
# legalcode_gu_list = []
# legalcode_dong_list = []
# admcode_code_list = []
# admcode_si_list = []
# admcode_gu_list = []
# admcode_dong_list = []
# addr_si_list = []
# addr_gu_list = []
# addr_dong_list = []
# addr_bunji1_list = []
# addr_bunji2_list = []
# roadaddr_si_list = []
# roadaddr_gu_list = []
# roadaddr_dong_list = []
# roadaddr_roadname_list = []
# roadaddr_detail_list = []

# # 데이터프레임의 각 행을 순회하면서 주소 정보를 가져와서 리스트에 추가
# for index, row in tqdm(park.iterrows()):
#     lat = row["Y좌표(WGS84)"]
#     lng = row["X좌표(WGS84)"]
#     legalcode_code, legalcode_si, legalcode_gu, legalcode_dong, admcode_code, admcode_si, admcode_gu, admcode_dong, addr_si, addr_gu, addr_dong, addr_bunji1, addr_bunji2, roadaddr_si, roadaddr_gu, roadaddr_dong, roadaddr_roadname, roadaddr_detail = reverse_geocoding(lat, lng, client_id, client_secret)
#     legalcode_code_list.append(legalcode_code)
#     legalcode_si_list.append(legalcode_si)
#     legalcode_gu_list.append(legalcode_gu)
#     legalcode_dong_list.append(legalcode_dong)
#     admcode_code_list.append(admcode_code)
#     admcode_si_list.append(admcode_si)
#     admcode_gu_list.append(admcode_gu)
#     admcode_dong_list.append(admcode_dong)
#     addr_si_list.append(addr_si)
#     addr_gu_list.append(addr_gu)
#     addr_dong_list.append(addr_dong)
#     addr_bunji1_list.append(addr_bunji1)
#     addr_bunji2_list.append(addr_bunji2)
#     roadaddr_si_list.append(roadaddr_si)
#     roadaddr_gu_list.append(roadaddr_gu)
#     roadaddr_dong_list.append(roadaddr_dong)
#     roadaddr_roadname_list.append(roadaddr_roadname)
#     roadaddr_detail_list.append(roadaddr_detail)

# # 새로운 열로 리스트를 데이터프레임에 추가
# park["법정동코드"] = legalcode_code_list
# park["법정동_시"] = legalcode_si_list
# park["법정동_구"] = legalcode_gu_list
# park["법정동_동"] = legalcode_dong_list

# park["행정동코드"] = admcode_code_list
# park["행정동_시"] = admcode_si_list
# park["행정동_구"] = admcode_gu_list
# park["행정동_동"] = admcode_dong_list

# park["지번주소_시"] = roadaddr_si_list
# park["지번주소_구"] = roadaddr_gu_list
# park["지번주소_동"] = roadaddr_dong_list
# park["지번주소_본번"] = roadaddr_roadname_list
# park["지번주소_부번"] = roadaddr_detail_list

# park["도로명주소_시"] = roadaddr_si_list
# park["도로명주소_구"] = roadaddr_gu_list
# park["도로명주소_동"] = roadaddr_dong_list
# park["도로명주소_도로명"] = roadaddr_roadname_list
# park["도로명주소_상세주소"] = roadaddr_detail_list

132it [00:09, 13.59it/s]


In [36]:
# 새로운 열을 추가할 빈 리스트 생성
words = []

# 주소 열을 순회하며 '동' 또는 '로'가 포함된 행을 추출하고 새로운 열에 해당 단어를 추가
for address in tqdm(park['공원주소']):
    if '동' in address or '가' in address:
        words.append(' '.join([word for word in address.split() if ('동' in word and '구' not in word) or ('가' in word and '구' not in word)]))
    else:
        words.append('')

# 추출한 단어를 담은 리스트를 새로운 열로 추가
park['공원주소_동'] = words

100%|████████████████████████████████████████████████████████████████████████████| 132/132 [00:00<00:00, 124667.45it/s]


In [55]:
park[(park['공원주소_동'] == '') & park['도로명주소_동'].isnull()]['공원주소']

2                  경기도 과천시 대공원광장로 102
19                 서울특별시 용산구 서빙고로 185
21         서울특별시 종로구 낙산길 41 (우) 03085
39        서울특별시 동대문구 답십리로 209(답십리공원) 
40     서울특별시 강남구 광평로10길 30-71(대모산공원) 
58       서울특별시 서대문구 통일로 507-22(백련공원) 
61                  서울특별시 은평구 구산동 산61
80       서울특별시 동작구 성대로6사길 27-6(상도공원) 
131                           서울로7017
Name: 공원주소, dtype: object

In [37]:
# 동 확인 어려운 경우 서칭하여 채워넣기
park.loc[1,'공원주소_동'] = '길동'
park.loc[13,'공원주소_동'] = '답십리동'
park.loc[15,'공원주소_동'] = '영등포동'
park.loc[16,'공원주소_동'] = '신대방동'
park.loc[21,'공원주소_동'] = '동숭동'
park.loc[31,'공원주소_동'] = '고척동'
park.loc[33,'공원주소_동'] = '가양동'
park.loc[34,'공원주소_동'] = '연희동'
park.loc[38,'공원주소_동'] = '대방동'
park.loc[40,'공원주소_동'] = '일원동'
park.loc[45,'공원주소_동'] = '신사동'
park.loc[46,'공원주소_동'] = '상월곡동'
park.loc[49,'공원주소_동'] = '동숭동'
park.loc[55,'공원주소_동'] = '전농동'
park.loc[61,'공원주소_동'] = '구산동'
park.loc[65,'공원주소_동'] = '명륜동'
park.loc[68,'공원주소_동'] = '정릉동'
park.loc[69,'공원주소_동'] = '합정동'
park.loc[71,'공원주소_동'] = '중계동'
park.loc[75,'공원주소_동'] = '사직동'
park.loc[80,'공원주소_동'] = '상도동'
park.loc[91,'공원주소_동'] = '신월동'
park.loc[95,'공원주소_동'] = '시흥동'
park.loc[97,'공원주소_동'] = '방학동'
park.loc[102,'공원주소_동'] = '금호동3가'
park.loc[105,'공원주소_동'] = '양재동'

park.loc[3,'공원주소_동'] = '성수동1가'
park.loc[10,'공원주소_동'] = '성수동2가'
park.loc[28,'공원주소_동'] = '사직동'

park.loc[130,'공원주소_동'] = '율현동'

park.loc[19,'공원주소_동'] = '용산동6가'
park.loc[39,'공원주소_동'] = '답십리동'
park.loc[58,'공원주소_동'] = '홍은동'
park.loc[131,'공원주소_동'] = '봉래동2가'

park.loc[4,'공원주소_동'] = '상암동'
park.loc[14,'공원주소_동'] = '을지로5가'
park.loc[18,'공원주소_동'] = '천호동'
park.loc[20,'공원주소_동'] = '현저동'
park.loc[23,'공원주소_동'] = '신림동'
park.loc[24,'공원주소_동'] = '효창동'
park.loc[41,'공원주소_동'] = '종로2가'
park.loc[50,'공원주소_동'] = '여의도동'
park.loc[53,'공원주소_동'] = '방화동'
park.loc[59,'공원주소_동'] = '오금동'
park.loc[73,'공원주소_동'] = '노량진동'
park.loc[78,'공원주소_동'] = '삼청동'
park.loc[86,'공원주소_동'] = '만리동2가'
park.loc[89,'공원주소_동'] = '도봉동'
park.loc[90,'공원주소_동'] = '번동'
park.loc[119,'공원주소_동'] = '봉래동2가'
park.loc[123,'공원주소_동'] = '갈현동'
park.loc[128,'공원주소_동'] = '성산동'

park.loc[0,'공원주소_동'] = '예장동'
park.loc[12,'공원주소_동'] = '여의도동'
park.loc[22,'공원주소_동'] = '양재동'
park.loc[26,'공원주소_동'] = '돈암동'
park.loc[54,'공원주소_동'] = '잠실동'
park.loc[64,'공원주소_동'] = '방이동'
park.loc[84,'공원주소_동'] = '잠실동'
park.loc[85,'공원주소_동'] = '당산동'
park.loc[109,'공원주소_동'] = '항동'

In [38]:
park = park[~(park['공원주소_동'] == '')]

In [39]:
park.columns

Index(['연번', '공원명', '공원개요', '면적', '개원일', '주요시설', '주요식물', '안내도', '오시는길',
       '이용시참고사항', '이미지', '지역', '공원주소', '관리부서', '전화번호', 'X좌표(GRS80TM)',
       'Y좌표(GRS80TM)', 'X좌표(WGS84)', 'Y좌표(WGS84)', '바로가기', '구', '법정동코드',
       '법정동_시', '법정동_구', '법정동_동', '행정동코드', '행정동_시', '행정동_구', '행정동_동', '지번주소_시',
       '지번주소_구', '지번주소_동', '지번주소_본번', '지번주소_부번', '도로명주소_시', '도로명주소_구',
       '도로명주소_동', '도로명주소_도로명', '도로명주소_상세주소', '공원주소_동'],
      dtype='object')

In [40]:
# 필요 데이터만 남기기
park_s = park[['연번', '공원명', '면적', '공원주소', '구', 'X좌표(GRS80TM)','Y좌표(GRS80TM)', 'X좌표(WGS84)', 'Y좌표(WGS84)', '법정동코드', '법정동_구', '법정동_동', '행정동코드', '행정동_구', '행정동_동', '지번주소_시', '지번주소_구', '지번주소_동', '지번주소_본번', '지번주소_부번', '도로명주소_시', '도로명주소_구', '도로명주소_동', '도로명주소_도로명', '도로명주소_상세주소', '공원주소_동']]

In [41]:
# 데이터 추가(주소)
park_s['지번주소'] = park_s['지번주소_시'] + ' ' + park_s['지번주소_구'] + ' ' + park_s['지번주소_동'] + ' ' + park_s['지번주소_본번'] + ' ' + park_s['지번주소_부번']
park_s['도로명주소'] = park_s['도로명주소_시'] + ' ' + park_s['도로명주소_구'] + ' ' + park_s['도로명주소_동'] + ' ' + park_s['도로명주소_도로명'] + ' ' + park_s['도로명주소_상세주소']

In [42]:
# 필요없는 column 제거
park_s.drop(['지번주소_시', '지번주소_구', '도로명주소_시', '도로명주소_구'], axis = 1, inplace=True)

In [43]:
park_s['도로명주소_동'] = park_s['도로명주소_동'].astype(str)

In [44]:
# 조건에 따라 값을 채워넣을 새로운 열 추가
park_s['동_최종'] = park_s.apply(lambda row: row['공원주소_동'] if row['도로명주소_동'].isspace() else row['도로명주소_동'], axis=1)

In [45]:
# 조건에 따라 값을 채워넣을 새로운 열 추가
park_s['동_최종'] = park_s.apply(lambda row: row['공원주소_동'] if row['동_최종'] == 'nan' else row['동_최종'], axis=1)

In [46]:
# 추가 데이터 처리

park_s[park_s['공원주소_동'].str.contains('진관내동')]['공원명']

park_s.loc[110,'공원주소_동'] = '진관동'

In [47]:
# 추가 데이터 처리
park_s.loc[park_s['연번'] == 1,'동_최종'] = '예장동'
park_s.loc[park_s['연번'] == 69,'구'] = '성북구'
park_s.loc[park_s['연번'] == 72,'동_최종'] = '상계동'
park_s.loc[park_s['연번'] == 82,'동_최종'] = '둔촌동'
park_s.loc[park_s['연번'] == 84,'동_최종'] = '상계동'
park_s.loc[park_s['연번'] == 111,'동_최종'] = '갈현동'
park_s.loc[park_s['연번'] == 125,'구'] = '중랑구'
park_s.loc[park_s['연번'] == 130,'구'] = '노원구'

park_s.loc[park_s['연번'] == 24,'동_최종'] = '봉천동'
park_s.loc[park_s['연번'] == 26,'동_최종'] = '독산동'

park_s = park_s.drop(park_s[park_s['연번'] == 132].index)

In [48]:
# 동별 공원 수 확인
park_s2 = park_s.groupby('동_최종')['동_최종'].count().reset_index(name='count')

park_count = pd.merge(park_s, park_s2, on=['동_최종'], how='left')

In [49]:
park_count['X좌표(WGS84)'] = park_count['X좌표(WGS84)'].astype('float32')
park_count['Y좌표(WGS84)'] = park_count['Y좌표(WGS84)'].astype('float32')

## 7) 학군 환경 요인

In [ ]:
sch_info.drop(columns=['Unnamed: 0'], inplace=True)
uni_info = pd.read_csv(args.default_path+'서울시 대학 및 전문대학.csv', encoding='cp949')

sch_info = sch_info.drop(columns=['관할조직명', '도로명우편번호', '도로명상세주소'], axis=1)

# 지번주소 시, 구, 동 나누기
sch_info['시'] = sch_info['지번주소'].str.split(' ').str[0].str.strip()
sch_info['구'] = sch_info['지번주소'].str.split(' ').str[1].str.strip()
sch_info['동'] = sch_info['지번주소'].str.split(' ').str[2].str.strip()

# 학교종류명 변경 (초, 중, 고, 특수, 외국인)
sch_info['학교종류명'] = sch_info['학교종류명'].replace({'평생학교(고)-2년6학기': '고등학교',
                                                        '각종학교(고)': '고등학교',
                                                        '평생학교(고)-3년6학기': '고등학교',
                                                        '각종학교(중)': '중학교',
                                                        '평생학교(중)-2년6학기': '중학교',
                                                        '방송통신고등학교': '고등학교',
                                                        '평생학교(초)-3년6학기': '초등학교',
                                                        '고등기술학교': '고등학교',
                                                        '방송통신중학교': '중학교',
                                                        '각종학교(초)': '초등학교',
                                                        '평생학교(중)-3년6학기': '중학교',
                                                        '공동실습소': '고등학교'})

# 설립구분 (국립, 사립, 공립) -> 국공립, 사립으로 변경
sch_info['설립구분'] = sch_info['설립구분'].replace({'국립':'국공립', '공립':'국공립'})

# 컬럼순서 변경
school_df = sch_info[['시', '구', '동', '학교명', '학교종류명', '설립구분', '도로명주소', '지번주소']]
school_df.head()

# 필요 컬럼만 사용
uni_info = uni_info[['행정시', '행정구', '행정동', '학교명', '학교종류', '설립', '주소', '지번주소']]

# 학교종류 전부 '대학교'로 통일
uni_info['학교종류'] = '대학교'

# 지번주소로 시, 구, 동 구분
uni_info['시'] = uni_info['지번주소'].str.split(' ').str[0].str.strip()
uni_info['구'] = uni_info['지번주소'].str.split(' ').str[1].str.strip()
uni_info['동'] = uni_info['지번주소'].str.split(' ').str[2].str.strip()

# 설립을 사립, 국공립으로 통일
uni_info['설립'] = uni_info['설립'].replace({'국립': '국공립', '공립': '국공립', '특별법국립': '국공립', '국립대법인': '국공립'})

# 위 초중고 정보에 맞춰 컬럼정보 일치
# '시' '구'	'동' '학교명' '학교종류명' '설립구분' '도로명주소' '지번주소'
uni_info.rename(columns={'학교종류':'학교종류명', '설립':'설립구분', '주소':'도로명주소'}, inplace=True)
university_df = uni_info[['시', '구', '동', '학교명', '학교종류명', '설립구분', '도로명주소', '지번주소']]

seoul_edu = pd.concat([school_df, university_df])
chk_df(seoul_edu)

In [105]:
# subway189_nan.to_csv('C:/python/Budongsan/new_data/subway189_nan.csv', encoding='utf-8')

### 좌표 붙인 뒤에

In [51]:
sch_info = pd.read_csv(args.default_path+'서울시_동별_초중고대_정보.csv', encoding='utf-8')

# 동별 학교 수 게싼
seoul_edu_df= sch_info.groupby(['시', '구', '동'])['동'].count().reset_index(name='총 학교 수')

# count 데이터 merge
seoul_edu_count = pd.merge(sch_info, seoul_edu_df, on=['시', '구', '동'], how='left')

chk_df(seoul_edu_count)

(1509, 11)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1509 entries, 0 to 1508
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   시       1509 non-null   object 
 1   구       1509 non-null   object 
 2   동       1509 non-null   object 
 3   학교명     1509 non-null   object 
 4   학교종류명   1509 non-null   object 
 5   설립구분    1509 non-null   object 
 6   도로명주소   1509 non-null   object 
 7   지번주소    1509 non-null   object 
 8   x       1509 non-null   float64
 9   y       1509 non-null   float64
 10  총 학교 수  1509 non-null   int64  
dtypes: float64(2), int64(1), object(8)
memory usage: 141.5+ KB
None
       시    구      동             학교명 학교종류명 설립구분                도로명주소  \
0  서울특별시  광진구     능동         선화예술중학교   중학교   사립   서울특별시 광진구 천호대로 664   
1  서울특별시  성동구  하왕십리동        서울숭신초등학교  초등학교  국공립    서울특별시 성동구 마장로 161   
2  서울특별시  성동구    행당동           행당중학교   중학교  국공립   서울특별시 성동구 왕십리로 189   
3  서울특별시  성동구    사근동  한양대학교사범대학부속중학교   중학교  

## 8)기타 요인

### 네이버 부동산 키워드 검색량(부동산, -매매, -정책)

In [52]:
naver_keyword = pd.read_csv(args.default_path + '부동산_매매_월별_키워드_검색어트렌드.csv')

# 컬럼명 변경
naver_keyword.rename(columns = {'날짜' : '시점'} , inplace = True)

# 일자 값 datetime으로 변경
chg_date(naver_keyword,'시점','%Y-%m-%d')

# '월' 열 추가
naver_keyword['연도'] = naver_keyword['시점'].dt.year
naver_keyword['월'] = naver_keyword['시점'].dt.month

# 불필요한 데이터 제거
naver_keyword.drop('시점',axis=1, inplace = True)

chk_df(naver_keyword)

(60, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   부동산     60 non-null     float64
 1   부동산 매매  60 non-null     float64
 2   부동산정책   60 non-null     float64
 3   연도      60 non-null     int64  
 4   월       60 non-null     int64  
dtypes: float64(3), int64(2)
memory usage: 2.5 KB
None
        부동산   부동산 매매    부동산정책    연도  월
0  70.27279  0.39200  2.49956  2018  1
1  57.24509  1.54207  0.20941  2018  2
2  65.12640  0.46358  0.21425  2018  3
3  55.54894  0.36836  0.19343  2018  4
4  55.92699  0.35808  0.17647  2018  5


### 부동산 114 상세정보

In [53]:
apt_detail = pd.read_csv(args.default_path + '아파트매매_크롤링_데이터정리_v1.csv')

# 불필요한 데이터 제거
apt_detail.drop('Unnamed: 0', axis=1 ,inplace = True)

# 불필요한 데이터 제거
apt_detail = apt_detail.drop([3595, 6342], axis = 0)

# 컬럼명 변경
apt_detail.rename(columns = {'구' : '자치구', '동' : '법정동', '번지수': '번지', '이름': '단지명'} , inplace = True)

chk_df(apt_detail)

(7673, 24)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7673 entries, 0 to 7674
Data columns (total 24 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   시        7673 non-null   object 
 1   자치구      7673 non-null   object 
 2   법정동      7673 non-null   object 
 3   번지       7673 non-null   object 
 4   유형       7673 non-null   object 
 5   단지명      7673 non-null   object 
 6   금액(만원)   7673 non-null   int64  
 7   전용률(%)   7673 non-null   float64
 8   방향       7673 non-null   object 
 9   공급면적(㎡)  7673 non-null   float64
 10  전용면적(㎡)  7673 non-null   float64
 11  매물층      7673 non-null   object 
 12  전체층      7673 non-null   object 
 13  방수       7673 non-null   int64  
 14  욕실수      7673 non-null   int64  
 15  아파트동수    7673 non-null   int64  
 16  총가구수     7673 non-null   int64  
 17  융자금      7673 non-null   object 
 18  주차대수     7673 non-null   int64  
 19  관리비      7673 non-null   int64  
 20  난방형태     7673 non-null   object 
 21  난방방

# 3. 데이터 최종 merge(master data 생성)

## 0) apt

In [54]:
apt_sale_price_main.columns

Index(['시군구', '번지', '본번', '부번', '단지명', '전용면적(㎡)', '시점', '계약일', '거래금액(만원)', '층',
       '건축년도', '도로명', '시', '자치구', '법정동', '연도', '월', '도로명주소', '지번주소', 'x', 'y'],
      dtype='object')

In [80]:
price_code.columns

Index(['시도코드', '시도명', '시군구코드', '시군구명', '행정동코드', '읍면동명', '법정동코드', '동리명', '생성일자',
       '말소일자', '시군구명_읍면동명'],
      dtype='object')

In [55]:
# key 값인 법정동코드 붙이기
apt_sale_price_main['자치구_법정동'] = apt_sale_price_main['자치구'] + ' ' + apt_sale_price_main['법정동']
price_code['시군구명_동리명'] = price_code['시군구명'] + ' ' + price_code['동리명']
apt_sale_price_main_s = apt_sale_price_main.join(price_code.set_index('시군구명_동리명')['법정동코드'], on='자치구_법정동', how='left')
apt_sale_price_main_s = apt_sale_price_main_s.reset_index(drop=True)
apt_sale_price_main_s = apt_sale_price_main_s[~apt_sale_price_main_s.duplicated()]

## 1) 인구사회 요인 merge

### - 인구이동

In [56]:
# key 값인 법정동코드와 합치기 위한 column 생성
pop_move['자치구_행정동'] = pop_move['자치구'] + ' ' + pop_move['행정동']
price_code['시군구명_읍면동명'] = price_code['시군구명'] + ' ' + price_code['읍면동명']

# merge를 위한 type 변경
pop_move['총전입'] = pop_move['총전입'].astype(int)
pop_move['총전출'] = pop_move['총전출'].astype(int)

# key 값인 법정동코드를 원데이터(인구이동)에 합치기
pop_move_s = pop_move.join(price_code.set_index('시군구명_읍면동명')['법정동코드'], on='자치구_행정동')
pop_move_s = pop_move_s.reset_index(drop=True)
pop_move_m = pop_move_s.groupby(['연도', '자치구', '행정동', '자치구_행정동'])['총전입', '총전출'].mean().reset_index()
pop_move_f = pd.merge(pop_move_m, pop_move_s, on=['연도', '자치구', '행정동', '자치구_행정동', '총전입', '총전출'])

### - 주민등록인구

In [57]:
# key 값인 법정동코드와 합치기 위한 column 생성
pop_ages['자치구_행정동'] = pop_ages['자치구'] + ' ' + pop_ages['행정동']

# merge를 위한 type 변경
pop_ages['주민등록인구수'] = pop_ages['주민등록인구수'].astype(int)

# key 값인 법정동코드를 원데이터(주민등록인구)에 합치기
pop_ages_s = pop_ages.join(price_code.set_index('시군구명_읍면동명')['법정동코드'], on='자치구_행정동')
pop_ages_s = pop_ages_s.reset_index(drop=True)
pop_ages_m = pop_ages_s.groupby(['연도', '자치구', '행정동', '자치구_행정동'])['주민등록인구수'].mean().reset_index()
pop_ages_f = pd.merge(pop_ages_m, pop_ages_s, on=['연도', '자치구', '행정동', '자치구_행정동', '주민등록인구수'])

### - 생활인구

In [93]:
life_people_mean.columns

Index(['시군구명', '연도', '월', '총생활인구수'], dtype='object')

### - merge 마무리

In [58]:
# merge를 위한 type 변경
apt_sale_price_main_s['연도'] = apt_sale_price_main_s['연도'].astype(int)
apt_sale_price_main_s['월'] = apt_sale_price_main_s['월'].astype(int)
apt_sale_price_main_s['법정동코드'] = apt_sale_price_main_s['법정동코드'].astype(int)
apt_sale_price_main_s['자치구'] = apt_sale_price_main_s['자치구'].astype(str)

pop_move_f['연도'] = pop_move_f['연도'].astype(int)
pop_move_f['법정동코드'] = pop_move_f['법정동코드'].astype(int)

pop_ages_f['연도'] = pop_ages_f['연도'].astype(int)
pop_ages_f['법정동코드'] = pop_ages_f['법정동코드'].astype(int)

life_people_mean['시군구명'] = life_people_mean['시군구명'].astype(str)
life_people_mean['연도'] = life_people_mean['연도'].astype(int)
life_people_mean['월'] = life_people_mean['월'].astype(int)

In [59]:
# 인구이동과 merge
apt_master = pd.merge(apt_sale_price_main_s, pop_move_f, left_on=['연도', '자치구', '법정동코드'], right_on=['연도', '자치구','법정동코드'], how='left')

apt_master = apt_master[~apt_master.duplicated(['시군구', '번지', '본번', '부번', '단지명', '전용면적(㎡)', '시점', '계약일', '거래금액(만원)', '층',
       '건축년도', '도로명', '시', '자치구', '법정동', '연도', '월', '도로명주소', '지번주소', 'x', 'y', '자치구_법정동'])]

apt_master.drop(['행정동', '자치구_행정동'], axis = 1, inplace=True)

In [60]:
# 주민등록인구와 merge
apt_master = pd.merge(apt_master, pop_ages_f, left_on=['연도', '자치구', '법정동코드'], right_on=['연도', '자치구','법정동코드'], how='left')

apt_master = apt_master[~apt_master.duplicated(['시군구', '번지', '본번', '부번', '단지명', '전용면적(㎡)', '시점', '계약일', '거래금액(만원)', '층',
       '건축년도', '도로명', '시', '자치구', '법정동', '연도', '월', '도로명주소', '지번주소', 'x', 'y', '자치구_법정동'])]

apt_master.drop(['행정동', '자치구_행정동'], axis = 1, inplace=True)

In [61]:
# 생활인구와 merge
apt_master = pd.merge(apt_master, life_people_mean, left_on=['자치구', '연도', '월'], right_on=['시군구명', '연도', '월'], how='left')

apt_master.drop(['시군구명'], axis = 1, inplace=True)

## 2) 교통 인프라 요인 merge

### - 서울시 버스정류장 현황 

In [62]:
# key 값인 법정동코드와 합치기 위한 column 생성
busstop_name_addr_count_main['자치구_법정동'] = busstop_name_addr_count_main['구'] + ' ' + busstop_name_addr_count_main['동']

# merge를 위한 type 변경
busstop_name_addr_count_main['count'] = busstop_name_addr_count_main['count'].astype(int)

# key 값인 법정동코드를 원데이터(버스정류장)에 합치기
busstop_name_addr_count_main_s = busstop_name_addr_count_main.join(price_code.set_index('시군구명_동리명')['법정동코드'], on='자치구_법정동')
busstop_name_addr_count_main_s = busstop_name_addr_count_main_s[~busstop_name_addr_count_main_s.duplicated()]
busstop_name_addr_count_main_s = busstop_name_addr_count_main_s.reset_index(drop=True)

### - 서울시 지하철역 정보 

In [63]:
# merge를 위한 type 변경
subway189_count['지하철역 수'] = subway189_count['지하철역 수'].astype(int)

# merge를 위한 기존 법정동코드 이름 변경
subway189_count.rename(columns={'법정동코드':'법정동코드_s'}, inplace=True)

# key 값인 법정동코드와 합치기 위한 column 생성
subway189_count['자치구_법정동'] = subway189_count['시군구명'] + ' ' + subway189_count['법정동명']

# key 값인 법정동코드를 원데이터(지하철역)에 합치기
subway189_count_s = subway189_count.join(price_code.set_index('시군구명_동리명')['법정동코드'], on='자치구_법정동')
subway189_count_s = subway189_count_s[~subway189_count_s.duplicated()]
subway189_count_s = subway189_count_s.reset_index(drop=True)

### - 도로율

In [101]:
road.columns

Index(['시군구', '연도', '도로율'], dtype='object')

### - merge 마무리

In [64]:
# merge를 위한 type 변경
busstop_name_addr_count_main_s['법정동코드'] = busstop_name_addr_count_main_s['법정동코드'].astype(int)

subway189_count_s['법정동코드'] = subway189_count_s['법정동코드'].astype(int)
subway189_count_s['자치구_법정동'] = subway189_count_s['자치구_법정동'].astype(str)
subway189_count_s['시군구명'] = subway189_count_s['시군구명'].astype(str)
subway189_count_s['법정동명'] = subway189_count_s['법정동명'].astype(str)

road['시군구'] = road['시군구'].astype(str)
road['연도'] = road['연도'].astype(int)

In [65]:
# 버스정류장과 merge
apt_master = pd.merge(apt_master, busstop_name_addr_count_main_s, left_on=['시', '자치구', '법정동', '자치구_법정동', '법정동코드'], right_on=['시', '구', '동', '자치구_법정동', '법정동코드'], how='left')

apt_master_s = apt_master[['단지명', '전용면적(㎡)', '자치구_법정동', '법정동코드', 'x_x', 'y_x', '정류소명', '시군구코드', '구', '동', '도로명코드', '도로명_y', '행정구역코드', 'x_y', 'y_y']]

In [66]:
%%time
# 두 위치 간의 거리 계산 함수
def calculate_distance(row):
    coords_1 = (row['y_x'], row['x_x'])
    coords_2 = (row['y_y'], row['x_y'])
    distance = haversine(coords_1, coords_2, unit=Unit.KILOMETERS)
    return distance

# 두 위치 간의 소요 시간 계산 함수
def calculate_duration(row):
    distance = calculate_distance(row)
    average_speed = 4  # km/h - 사람 평균 걷는 속도
    duration = distance / average_speed
    return duration

# 거리와 소요 시간을 계산하여 새 변수에 추가
apt_master_s['버스정류장_거리'] = apt_master_s.apply(calculate_distance, axis=1)
apt_master_s['버스정류장_소요시간'] = apt_master_s.apply(calculate_duration, axis=1)

CPU times: total: 18min 14s
Wall time: 24min 21s


In [67]:
# 기준컬럼 리스트 설정
groupList = ['자치구_법정동', '법정동코드']

# 집계/통계 내고자하는 컬럼 설정
y = ['버스정류장_거리', '버스정류장_소요시간']

# 통계내어 데이터프레임을 만듦
apt_master_mean = apt_master_s.groupby(groupList)[y].agg('mean').reset_index()
apt_master_mean

,자치구_법정동,법정동코드,버스정류장_거리,버스정류장_소요시간
0,강남구 개포동,1168010300,1.429190,0.357297
1,강남구 논현동,1168010800,0.700189,0.175047
2,강남구 대치동,1168010600,0.740984,0.185246
3,강남구 도곡동,1168011800,0.779499,0.194875
4,강남구 삼성동,1168010500,0.743394,0.185848
...,...,...,...,...
331,중랑구 면목동,1126010100,1.209060,0.302265
332,중랑구 묵동,1126010400,0.666307,0.166577
333,중랑구 상봉동,1126010200,0.829670,0.207417
334,중랑구 신내동,1126010600,1.308995,0.327249


In [68]:
apt_master = apt_master[~apt_master.duplicated(['시군구', '번지', '본번', '부번', '단지명', '전용면적(㎡)', '시점', '계약일', '거래금액(만원)', '층',
       '건축년도', '도로명_x', '시', '자치구', '법정동', '연도', '월', '도로명주소_x', '지번주소_x', 'x_x', 'y_x', '자치구_법정동', '법정동코드'])]

In [69]:
apt_master = pd.merge(apt_master, apt_master_mean, left_on=['자치구_법정동', '법정동코드'], right_on=['자치구_법정동', '법정동코드'], how='left')

In [70]:
apt_master.drop(['시군구코드', '구', '동', '도로명코드', '행정구역코드', '도로명주소_y', '지번주소_y', 'x_y', 'y_y'], axis = 1, inplace=True)

In [71]:
apt_master.rename(columns={'도로명_x':'도로명_apt', '도로명_y':'도로명_bus', 'count':'버스정류장수'}, inplace=True)

In [72]:
apt_master = apt_master.reset_index(drop=True)

In [74]:
# 지하철역과 merge
apt_master = pd.merge(apt_master, subway189_count_s, left_on=['자치구', '법정동', '자치구_법정동', '법정동코드'], right_on=['시군구명', '법정동명', '자치구_법정동', '법정동코드'], how='left')

In [79]:
apt_master_s = apt_master[['단지명', '전용면적(㎡)', '자치구_법정동', '법정동코드', 'x_x', 'y_x', '정류소명', '시군구명', '법정동명', '도로명주소', '역 주소(지번주소)', '역사명', '역 위치(경도)', '역 위치(위도)']]

In [80]:
%%time
# 두 위치 간의 거리 계산 함수
def calculate_distance(row):
    coords_1 = (row['y_x'], row['x_x'])
    coords_2 = (row['역 위치(위도)'], row['역 위치(경도)'])
    distance = haversine(coords_1, coords_2, unit=Unit.KILOMETERS)
    return distance

# 두 위치 간의 소요 시간 계산 함수
def calculate_duration(row):
    distance = calculate_distance(row)
    average_speed = 4  # km/h - 사람 평균 걷는 속도
    duration = distance / average_speed
    return duration

# 거리와 소요 시간을 계산하여 새 변수에 추가
apt_master_s['지하철역_거리'] = apt_master_s.apply(calculate_distance, axis=1)
apt_master_s['지하철역_소요시간'] = apt_master_s.apply(calculate_duration, axis=1)

CPU times: total: 38 s
Wall time: 42.4 s


In [81]:
# 기준컬럼 리스트 설정
groupList = ['자치구_법정동', '법정동코드']

# 집계/통계 내고자하는 컬럼 설정
y = ['지하철역_거리', '지하철역_소요시간']

# 통계내어 데이터프레임을 만듦
apt_master_mean = apt_master_s.groupby(groupList)[y].agg('mean').reset_index()
apt_master_mean

,자치구_법정동,법정동코드,지하철역_거리,지하철역_소요시간
0,강남구 개포동,1168010300,1.002912,0.250728
1,강남구 논현동,1168010800,0.884449,0.221112
2,강남구 대치동,1168010600,0.909739,0.227435
3,강남구 도곡동,1168011800,0.761786,0.190446
4,강남구 삼성동,1168010500,1.053141,0.263285
...,...,...,...,...
331,중랑구 면목동,1126010100,1.045014,0.261253
332,중랑구 묵동,1126010400,0.692441,0.173110
333,중랑구 상봉동,1126010200,0.627176,0.156794
334,중랑구 신내동,1126010600,1.305855,0.326464


In [82]:
apt_master = apt_master[~apt_master.duplicated(['시군구', '번지', '본번', '부번', '단지명', '전용면적(㎡)', '시점', '계약일', '거래금액(만원)', '층',
       '건축년도', '도로명_apt', '시', '자치구', '법정동', '연도', '월', '도로명주소_x', '지번주소_x', 'x_x', 'y_x', '자치구_법정동', '법정동코드'])]

In [83]:
apt_master = pd.merge(apt_master, apt_master_mean, left_on=['자치구_법정동', '법정동코드'], right_on=['자치구_법정동', '법정동코드'], how='left')

In [84]:
apt_master.drop(['역 위치(경도)', '역 위치(위도)', '역 주소(지번주소)', '도로명주소', '시군구명', '법정동명', '법정동코드_s'], axis = 1, inplace=True)

In [88]:
# 도로율과 merge
apt_master = pd.merge(apt_master, road, left_on=['자치구', '연도'], right_on=['시군구', '연도'], how='left')
apt_master.rename(columns={'시군구_x':'시군구', 'x_x':'x', 'y_x':'y'}, inplace=True)
apt_master = apt_master.reset_index(drop=True)
apt_master.drop(['시군구_y'], axis = 1, inplace=True)

## 3) 부동산 주거 시장 요인 merge

### - 지역 내 총생산

In [89]:
grdp.columns

Index(['자치구', '연도', '지역내총생산'], dtype='object')

### - 지역별 월평균 소득 

In [90]:
gri.columns

Index(['연도', '행정동코드', '행정동', '월평균소득금액'], dtype='object')

In [92]:
# merge를 위한 type 변경
gri['행정동코드'] = gri['행정동코드'].astype(str)
gri['행정동코드'] = gri['행정동코드'] + '00'
gri['행정동코드'] = gri['행정동코드'].astype(int)

gri['월평균소득금액'] = gri['월평균소득금액'].astype(int)

# key 값인 법정동코드를 원데이터(지역별 월평균 소득)에 합치기
gri_s = gri.join(price_code.set_index('행정동코드')['법정동코드'], on='행정동코드')
gri_s = gri_s.reset_index(drop=True)

### - 기준금리 

In [96]:
ir.columns

Index(['연도', '기준금리'], dtype='object')

### - 소비자물가지수

In [97]:
cpi_mean.columns

Index(['연도', '월', '소비자물가지수'], dtype='object')

### - 전세가율

In [98]:
sale_jeonse_rt.columns

Index(['자치구', '시점', '전세가율', '연도', '월'], dtype='object')

### - 전월세전환율

In [99]:
rent_conversion_rt_apt.columns

Index(['자치구', '시점', '전월세전환율', '연도', '월'], dtype='object')

### - 지가지수

In [100]:
LPI_m.columns

Index(['자치구', '시점', '지가지수', '연도', '월'], dtype='object')

### - 실거래가격지수

In [101]:
PI_apt.columns

Index(['시점', '실거래가격지수', '연도', '월'], dtype='object')

### - 소비심리지수

In [102]:
CSI.columns

Index(['시점', '소비심리지수', '연도', '월'], dtype='object')

### - merge 마무리

In [103]:
# merge를 위한 type 변경
grdp['자치구'] = grdp['자치구'].astype(str)
grdp['연도'] = grdp['연도'].astype(int)

gri_s['연도'] = gri_s['연도'].astype(int)
gri_s['법정동코드'] = gri_s['법정동코드'].astype(int)

ir['연도'] = ir['연도'].astype(int)

cpi_mean['연도'] = cpi_mean['연도'].astype(int)
cpi_mean['월'] = cpi_mean['월'].astype(int)

sale_jeonse_rt['자치구'] = sale_jeonse_rt['자치구'].astype(str)
sale_jeonse_rt['연도'] = sale_jeonse_rt['연도'].astype(int)
sale_jeonse_rt['월'] = sale_jeonse_rt['월'].astype(int)

rent_conversion_rt_apt['자치구'] = rent_conversion_rt_apt['자치구'].astype(str)
rent_conversion_rt_apt['연도'] = rent_conversion_rt_apt['연도'].astype(int)
rent_conversion_rt_apt['월'] = rent_conversion_rt_apt['월'].astype(int)

LPI_m['자치구'] = LPI_m['자치구'].astype(str)
LPI_m['연도'] = LPI_m['연도'].astype(int)
LPI_m['월'] = LPI_m['월'].astype(int)

PI_apt['연도'] = PI_apt['연도'].astype(int)
PI_apt['월'] = PI_apt['월'].astype(int)

CSI['연도'] = CSI['연도'].astype(int)
CSI['월'] = CSI['월'].astype(int)

In [104]:
# 지역내총생산 merge
apt_master = pd.merge(apt_master, grdp, left_on=['자치구', '연도'], right_on=['자치구', '연도'], how='left')

In [105]:
# 지역별 월평균 소득 merge
apt_master = pd.merge(apt_master, gri_s, left_on=['연도', '법정동코드'], right_on=['연도', '법정동코드'], how='left')

apt_master = apt_master[~apt_master.duplicated(['시군구', '번지', '본번', '부번', '단지명', '전용면적(㎡)', '시점', '계약일', '거래금액(만원)', '층',
       '건축년도', '도로명_apt', '시', '자치구', '법정동', '연도', '월', '도로명주소_x', '지번주소_x', 'x', 'y', '자치구_법정동', '법정동코드'])]

apt_master.drop(['행정동코드', '행정동'], axis = 1, inplace=True)

apt_master = apt_master.reset_index(drop=True)

In [106]:
# 기준금리 merge
apt_master = pd.merge(apt_master, ir, left_on=['연도'], right_on=['연도'], how='left')

In [107]:
# 소비자물가지수 merge
apt_master = pd.merge(apt_master, cpi_mean, left_on=['연도', '월'], right_on=['연도', '월'], how='left')

In [111]:
# 전세가율 merge
apt_master = pd.merge(apt_master, sale_jeonse_rt, left_on=['자치구', '연도', '월'], right_on=['자치구', '연도', '월'], how='left')
apt_master = apt_master.reset_index(drop=True)
apt_master.drop(['시점_y'], axis = 1, inplace=True)
apt_master.rename(columns={'시점_x': '시점'}, inplace=True)

In [112]:
# 전월세전환율 merge
apt_master = pd.merge(apt_master, rent_conversion_rt_apt, left_on=['자치구', '연도', '월'], right_on=['자치구', '연도', '월'], how='left')
apt_master = apt_master.reset_index(drop=True)
apt_master.drop(['시점_y'], axis = 1, inplace=True)
apt_master.rename(columns={'시점_x': '시점'}, inplace=True)

In [113]:
# 지가지수 merge
apt_master = pd.merge(apt_master, LPI_m, left_on=['자치구', '연도', '월'], right_on=['자치구', '연도', '월'], how='left')
apt_master = apt_master.reset_index(drop=True)
apt_master.drop(['시점_y'], axis = 1, inplace=True)
apt_master.rename(columns={'시점_x': '시점'}, inplace=True)

In [114]:
# 실거래가격지수 merge
apt_master = pd.merge(apt_master, PI_apt, left_on=['연도', '월'], right_on=['연도', '월'], how='left')
apt_master = apt_master.reset_index(drop=True)
apt_master.drop(['시점_y'], axis = 1, inplace=True)
apt_master.rename(columns={'시점_x': '시점'}, inplace=True)

In [118]:
# 소비심리지수 merge
apt_master = pd.merge(apt_master, CSI, left_on=['연도', '월'], right_on=['연도', '월'], how='left')
apt_master = apt_master.reset_index(drop=True)
apt_master.drop(['시점_y'], axis = 1, inplace=True)
apt_master.rename(columns={'시점_x': '시점'}, inplace=True)

## 4) 주거 환경 요인 merge

### - 상권(편의점, 대형마트, 백화점, 음식점, 카페(프렌차이즈), 병원) 수

In [120]:
# key 값인 법정동코드와 합치기 위한 column 생성
comm_district['자치구_법정동'] = comm_district['자치구'] + ' ' + comm_district['법정동명']

# key 값인 법정동코드를 원데이터(상권 수)에 합치기
comm_district_s = comm_district.join(price_code.set_index('시군구명_동리명')['법정동코드'], on='자치구_법정동')
comm_district_s = comm_district_s[~comm_district_s.duplicated()]
comm_district_s = comm_district_s.reset_index(drop=True)

### - 5대 범죄 발생 현황

In [121]:
crime.columns

Index(['자치구', '연도', '5대범죄(총)'], dtype='object')

### - merge 마무리

In [119]:
# merge를 위한 type 변경
comm_district['자치구'] = comm_district['자치구'].astype(str)
comm_district['법정동명'] = comm_district['법정동명'].astype(str)

crime['자치구'] = crime['자치구'].astype(str)
crime['연도'] = crime['연도'].astype(int)

In [122]:
# 상권 수와 merge
apt_master = pd.merge(apt_master, comm_district_s, left_on=['자치구', '법정동', '자치구_법정동', '법정동코드'], right_on=['자치구', '법정동명', '자치구_법정동', '법정동코드'], how='left')
apt_master.drop(['법정동명'], axis = 1, inplace=True)

In [122]:
# 범죄 현황과 merge
apt_master = pd.merge(apt_master, crime, left_on=['자치구', '연도'], right_on=['자치구', '연도'], how='left')
apt_master = apt_master.reset_index(drop=True)

In [124]:
apt_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292465 entries, 0 to 292464
Data columns (total 54 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   시군구         292465 non-null  object 
 1   번지          292358 non-null  object 
 2   본번          292422 non-null  float64
 3   부번          292422 non-null  float64
 4   단지명         292465 non-null  object 
 5   전용면적(㎡)     292465 non-null  float64
 6   시점          292465 non-null  object 
 7   계약일         292465 non-null  int64  
 8   거래금액(만원)    292465 non-null  object 
 9   층           292465 non-null  int64  
 10  건축년도        292463 non-null  float64
 11  도로명_apt     292372 non-null  object 
 12  시           292465 non-null  object 
 13  자치구         292465 non-null  object 
 14  법정동         292465 non-null  object 
 15  연도          292465 non-null  int32  
 16  월           292465 non-null  int32  
 17  도로명주소_x     292465 non-null  object 
 18  지번주소_x      292358 non-null  object 
 19  x 

## 5) 일자리 인프라 요인 merge

### - 경제활동인구

In [128]:
economy_act.columns

Index(['시도별', '시점', '경제활동인구 (천명)'], dtype='object')

### - 시군구별 고용보험 피보험자 수

In [132]:
employment_insurance.columns

Index(['시군구', '연도', '시', '구', '피보험자수'], dtype='object')

### - 지역별 사업체수/사업 유형 
-> 사업체는 별도 처리가 필요했어서 따로 추가 예정

In [261]:
# business_big_wide_real_final.columns

# # key 값인 법정동코드와 합치기 위한 column 생성
# comm_district['자치구_법정동'] = comm_district['자치구'] + ' ' + comm_district['법정동명']

# # key 값인 법정동코드를 원데이터(상권 수)에 합치기
# comm_district_s = comm_district.join(price_code.set_index('시군구명_동리명')['법정동코드'], on='자치구_법정동')
# comm_district_s = comm_district_s[~comm_district_s.duplicated()]
# comm_district_s = comm_district_s.reset_index(drop=True)

# # key 값인 법정동코드와 합치기 위한 column 생성
# business_big_wide_real_final['자치구_법정동'] = business_big_wide_real_final['구'] + ' ' + business_big_wide_real_final['법정동']

# business_big_wide_real_final_s = business_big_wide_real_final.join(price_code.set_index('시군구명_동리명')['법정동코드'], on='자치구_법정동')

# business_big_wide_real_final_s[~business_big_wide_real_final_s['법정동코드'].isnull()]

# business_big_wide_s2 = business_big_wide_s[business_big_wide_s['법정동코드'].isnull()]
# business_big_wide_s2 = business_big_wide_s2.drop('법정동코드', axis=1)

# business_big_wide_s2 = business_big_wide_s2.join(price_code.set_index('시군구명_읍면동명')['법정동코드'], on='자치구_법정동')

# business_big_wide_s2.info()

# business_big_wide_s2.columns

# business_big_wide_m = business_big_wide_s2.groupby(['구', '동', '자치구_법정동'])['건설업', '공공 행정, 국방 및 사회보장 행정', '광업', '교육 서비스업', '금융 및 보험업',
#        '농업, 임업 및 어업', '도매 및 소매업', '보건업 및 사회복지 서비스업', '부동산업',
#        '사업시설 관리, 사업 지원 및 임대 서비스업', '수도, 하수 및 폐기물 처리, 원료 재생업', '숙박 및 음식점업',
#        '예술, 스포츠 및 여가관련 서비스업', '운수 및 창고업', '전기, 가스, 증기 및 공기 조절 공급업',
#        '전문, 과학 및 기술 서비스업', '정보통신업', '제조업', '협회 및 단체, 수리 및 기타 개인 서비스업'].mean().reset_index()

# business_big_wide_m

# business_big_wide_s2.drop(['건설업', '공공 행정, 국방 및 사회보장 행정', '광업', '교육 서비스업', '금융 및 보험업',
#        '농업, 임업 및 어업', '도매 및 소매업', '보건업 및 사회복지 서비스업', '부동산업',
#        '사업시설 관리, 사업 지원 및 임대 서비스업', '수도, 하수 및 폐기물 처리, 원료 재생업', '숙박 및 음식점업',
#        '예술, 스포츠 및 여가관련 서비스업', '운수 및 창고업', '전기, 가스, 증기 및 공기 조절 공급업',
#        '전문, 과학 및 기술 서비스업', '정보통신업', '제조업', '협회 및 단체, 수리 및 기타 개인 서비스업'], axis = 1, inplace=True)

# business_big_wide_f = pd.merge(business_big_wide_s2, business_big_wide_m, on=['구', '동', '자치구_법정동'])
# business_big_wide_f

# business_big_wide_final = pd.concat([business_big_wide_s3, business_big_wide_f])

# business_big_wide_final

# business_big_wide_final = business_big_wide_final.reset_index(drop=True)

# business_big_wide_final.info()

### - merge 마무리

In [139]:
# merge를 위한 type 변경
economy_act['시점'] = economy_act['시점'].astype(int)

employment_insurance['연도'] = employment_insurance['연도'].astype(int)
employment_insurance['시'] = employment_insurance['시'].astype(str)
employment_insurance['구'] = employment_insurance['구'].astype(str)

# business_big_wide_real_final['구'] = business_big_wide_real_final['구'].astype(str)
# business_big_wide_real_final['법정동'] = business_big_wide_real_final['법정동'].astype(str)

In [142]:
# 경제활동과 merge
apt_master = pd.merge(apt_master, economy_act, left_on=['연도'], right_on=['시점'], how='left')
apt_master = apt_master.reset_index(drop=True)
apt_master.drop(['시도별', '시점_y'], axis = 1, inplace=True)
apt_master.rename(columns={'시점_x': '시점'}, inplace=True)

In [143]:
# 피보험자 수 merge
apt_master = pd.merge(apt_master, employment_insurance, left_on=['연도', '시', '자치구'], right_on=['연도', '시', '구'], how='left')
apt_master = apt_master.reset_index(drop=True)
apt_master.drop(['시군구_y', '구'], axis = 1, inplace=True)
apt_master.rename(columns={'시군구_x': '시군구'}, inplace=True)

## 6) 자연 환경 요인 merge

### - 서울시 주요 공원 현황

In [144]:
# 기존 데이터의 법정동코드 잠시 변경
park_count.rename(columns = {'법정동코드':'법정동코드_s'}, inplace=True)

# key 값인 법정동코드와 합치기 위한 column 생성
park_count['자치구_법정동'] = park_count['구'] + ' ' + park_count['동_최종']

# key 값인 법정동코드를 원데이터(상권 수)에 합치기
park_count_s = park_count.join(price_code.set_index('시군구명_동리명')['법정동코드'], on='자치구_법정동')
park_count_s = park_count_s.reset_index(drop=True)

# count를 공원 수로 변경
park_count_s.rename(columns={'count': '공원수'}, inplace=True)

### - merge 마무리

In [145]:
# merge를 위한 type 변경
park_count_s['법정동코드'] = park_count_s['법정동코드'].astype(int)

In [146]:
# 공원수와 merge
apt_master = pd.merge(apt_master, park_count_s, left_on=['법정동코드'], right_on=['법정동코드'], how='left')

In [147]:
%%time
# 두 위치 간의 거리 계산 함수
def calculate_distance(row):
    coords_1 = (row['y'], row['x'])
    coords_2 = (row['Y좌표(WGS84)'], row['X좌표(WGS84)'])
    distance = haversine(coords_1, coords_2, unit=Unit.KILOMETERS)
    return distance

# 두 위치 간의 소요 시간 계산 함수
def calculate_duration(row):
    distance = calculate_distance(row)
    average_speed = 4  # km/h - 사람 평균 걷는 속도
    duration = distance / average_speed
    return duration

# 거리와 소요 시간을 계산하여 새 변수에 추가
apt_master['공원_거리'] = apt_master.apply(calculate_distance, axis=1)
apt_master['공원_소요시간'] = apt_master.apply(calculate_duration, axis=1)

CPU times: total: 1min 56s
Wall time: 2min 11s


In [148]:
# 기준컬럼 리스트 설정
groupList = ['자치구_법정동_x', '법정동코드']

# 집계/통계 내고자하는 컬럼 설정
y = ['공원_거리', '공원_소요시간']

# 통계내어 데이터프레임을 만듦
apt_master_mean = apt_master.groupby(groupList)[y].agg('mean').reset_index()
apt_master_mean

,자치구_법정동_x,법정동코드,공원_거리,공원_소요시간
0,강남구 개포동,1168010300,NaN,NaN
1,강남구 논현동,1168010800,NaN,NaN
2,강남구 대치동,1168010600,NaN,NaN
3,강남구 도곡동,1168011800,0.630776,0.157694
4,강남구 삼성동,1168010500,0.782378,0.195594
...,...,...,...,...
331,중랑구 면목동,1126010100,1.417465,0.354366
332,중랑구 묵동,1126010400,1.185007,0.296252
333,중랑구 상봉동,1126010200,NaN,NaN
334,중랑구 신내동,1126010600,NaN,NaN


In [149]:
apt_master = apt_master[~apt_master.duplicated(['시군구', '번지', '본번', '부번', '단지명', '전용면적(㎡)', '시점', '계약일', '거래금액(만원)', '층',
       '건축년도', '도로명_apt', '시', '자치구', '법정동', '연도', '월', '도로명주소_x', '지번주소_x',
       'x', 'y', '자치구_법정동_x', '법정동코드'])]

In [150]:
apt_master = pd.merge(apt_master, apt_master_mean, left_on=['자치구_법정동_x', '법정동코드'], right_on=['자치구_법정동_x', '법정동코드'], how='left')

In [151]:
apt_master.drop(['연번', '공원명', '면적', '공원주소', '구', '법정동코드_s', 'X좌표(GRS80TM)', 'Y좌표(GRS80TM)', 'X좌표(WGS84)', 'Y좌표(WGS84)', '법정동코드_s', '법정동_구', '법정동_동', '행정동코드', '행정동_구', '행정동_동', '지번주소_동', '지번주소_본번', '지번주소_부번', '도로명주소_동', '도로명주소_도로명', '도로명주소_상세주소', '공원주소_동', '지번주소', '도로명주소', '동_최종','자치구_법정동_y', '공원_거리_y', '공원_소요시간_y'], axis = 1, inplace=True)

In [152]:
apt_master.rename(columns={'도로명주소_x': '도로명주소', '지번주소_x': '지번주소', '자치구_법정동_x': '자치구_법정동', '공원_거리_x': '공원_거리', '공원_소요시간_x': '공원_소요시간'}, inplace=True)

## 7) 학군 환경 요인 merge

In [153]:
# key 값인 법정동코드와 합치기 위한 column 생성
seoul_edu_count['자치구_법정동'] = seoul_edu_count['구'] + ' ' + seoul_edu_count['동']

# merge를 위한 type 변경
seoul_edu_count['총 학교 수'] = seoul_edu_count['총 학교 수'].astype(int)

# key 값인 법정동코드를 원데이터(총 학교 수)에 합치기
seoul_edu_count_s = seoul_edu_count.join(price_code.set_index('시군구명_동리명')['법정동코드'], on='자치구_법정동')
seoul_edu_count_s = seoul_edu_count_s[~seoul_edu_count_s.duplicated()]
seoul_edu_count_s = seoul_edu_count_s.reset_index(drop=True)

### - merge 마무리

In [154]:
# merge를 위한 type 변경
seoul_edu_count_s['법정동코드'] = seoul_edu_count_s['법정동코드'].astype(int)

In [155]:
# 학교 수와 merge
apt_master = pd.merge(apt_master, seoul_edu_count_s, left_on=['시', '자치구', '법정동', '자치구_법정동', '법정동코드'], right_on=['시', '구', '동', '자치구_법정동', '법정동코드'], how='left')

In [156]:
apt_master_s = apt_master[['단지명', '전용면적(㎡)', '자치구_법정동', '법정동코드', 'x_x', 'y_x', '학교명', '구', '동', '도로명주소_y', '지번주소_y', 'x_y', 'y_y', '총 학교 수']]

In [157]:
%%time
# 두 위치 간의 거리 계산 함수
def calculate_distance(row):
    coords_1 = (row['y_x'], row['x_x'])
    coords_2 = (row['y_y'], row['x_y'])
    distance = haversine(coords_1, coords_2, unit=Unit.KILOMETERS)
    return distance

# 두 위치 간의 소요 시간 계산 함수
def calculate_duration(row):
    distance = calculate_distance(row)
    average_speed = 4  # km/h - 사람 평균 걷는 속도
    duration = distance / average_speed
    return duration

# 거리와 소요 시간을 계산하여 새 변수에 추가
apt_master_s['학교_거리'] = apt_master_s.apply(calculate_distance, axis=1)
apt_master_s['학교_소요시간'] = apt_master_s.apply(calculate_duration, axis=1)

CPU times: total: 2min 55s
Wall time: 3min 18s


In [158]:
# 기준컬럼 리스트 설정
groupList = ['자치구_법정동', '법정동코드']

# 집계/통계 내고자하는 컬럼 설정
y = ['학교_거리', '학교_소요시간']

# 통계내어 데이터프레임을 만듦
apt_master_mean = apt_master_s.groupby(groupList)[y].agg('mean').reset_index()
apt_master_mean

,자치구_법정동,법정동코드,학교_거리,학교_소요시간
0,강남구 개포동,1168010300,1.223331,0.305833
1,강남구 논현동,1168010800,0.852277,0.213069
2,강남구 대치동,1168010600,0.871290,0.217822
3,강남구 도곡동,1168011800,0.850827,0.212707
4,강남구 삼성동,1168010500,0.880871,0.220218
...,...,...,...,...
331,중랑구 면목동,1126010100,1.182115,0.295529
332,중랑구 묵동,1126010400,0.717761,0.179440
333,중랑구 상봉동,1126010200,0.655662,0.163915
334,중랑구 신내동,1126010600,0.950550,0.237637


In [159]:
apt_master = apt_master[~apt_master.duplicated(['시군구', '번지', '본번', '부번', '단지명', '전용면적(㎡)', '시점', '계약일', '거래금액(만원)', '층',
       '건축년도', '도로명_apt', '시', '자치구', '법정동', '연도', '월', '도로명주소_x', '지번주소_x',
       'x_x', 'y_x', '자치구_법정동', '법정동코드'])]

In [160]:
apt_master = pd.merge(apt_master, apt_master_mean, left_on=['자치구_법정동', '법정동코드'], right_on=['자치구_법정동', '법정동코드'], how='left')

In [161]:
apt_master.drop(['구', '동', '학교종류명', '설립구분', '도로명주소_y', '지번주소_y', 'x_y', 'y_y'], axis = 1, inplace=True)

In [162]:
apt_master.rename(columns={'도로명주소_x':'도로명주소', '지번주소_x':'지번주소', 'x_x':'x', 'y_x':'y', '총 학교 수':'학교수'}, inplace=True)

In [163]:
apt_master = apt_master.reset_index(drop=True)

## 8) 기타 요인

### - 네이버 부동산 키워드 검색량(부동산, -매매, -정책)

In [164]:
naver_keyword.columns

Index(['부동산', '부동산 매매', '부동산정책', '연도', '월'], dtype='object')

### - 부동산 114 상세정보 - 분석에선 제외

In [165]:
apt_detail.columns

Index(['시', '자치구', '법정동', '번지', '유형', '단지명', '금액(만원)', '전용률(%)', '방향',
       '공급면적(㎡)', '전용면적(㎡)', '매물층', '전체층', '방수', '욕실수', '아파트동수', '총가구수', '융자금',
       '주차대수', '관리비', '난방형태', '난방방식', '최초입주년도', '건설회사'],
      dtype='object')

In [352]:
# apt_detail['자치구_법정동'] = apt_detail['자치구'] + ' ' + apt_detail['법정동']

# apt_detail_s = apt_detail.join(price_code.set_index('시군구명_동리명')['법정동코드'], on='자치구_법정동')

# apt_detail_s = apt_detail_s[~apt_detail_s.duplicated()]

# apt_detail_s = apt_detail_s.reset_index(drop=True)

### - merge 마무리

In [166]:
# merge를 위한 type 변경
naver_keyword['연도'] = naver_keyword['연도'].astype(int)
naver_keyword['월'] = naver_keyword['월'].astype(int)

# apt_detail_s['법정동코드'] = apt_detail_s['법정동코드'].astype(int)

In [168]:
# 네이버 키워드 merge
apt_master = pd.merge(apt_master, naver_keyword, left_on=['연도', '월'], right_on=['연도', '월'], how='left')

In [169]:
apt_master.to_csv('apt_master_20230730.csv', encoding='utf-8')

In [608]:
# apt_master = pd.merge(apt_master, apt_detail_s, left_on=['시', '자치구', '법정동', '자치구_법정동', '법정동코드', '단지명'], right_on=['시', '자치구', '법정동', '자치구_법정동', '법정동코드', '단지명'], how='left')

# apt_detail_s['번지'].value_counts()

# apt_detail_s['법정동'].value_counts()

# apt_master[apt_master['거래금액(만원)'].notnull()]['법정동'].value_counts()

# apt_master = apt_master[~apt_master.duplicated(['시군구', '번지', '본번', '부번', '단지명', '전용면적(㎡)', '시점', '계약일', '거래금액(만원)', '층',
#        '건축년도', '도로명_apt', '시', '자치구', '법정동', '연도', '월', '도로명주소_x', '자치구_법정동'])]

# apt_master.drop(['구', '동', '학교명', '학교종류명', '설립구분', '도로명주소_y', '지번주소', '도로명주소_api', '지번주소_api', '거리', '우편번호'], axis = 1, inplace=True)

# apt_master.rename(columns={'도로명주소_x':'도로명주소', '총 학교 수':'학교수'}, inplace=True)

# apt_master = apt_master.reset_index(drop=True)